## **Imports**

In [3]:
import pandas as pd
import numpy as np
import importlib
from application.modeling import disbursements
from application.modeling import interest_income
from application.modeling import other_income
from application.modeling import helper
from application.modeling import depreciation
from application.modeling import expenses
from application.modeling import income_statement
from application.modeling import direct_cashflow
from application.modeling import balance_sheet
from application.modeling import statement_of_cashflows
from application.modeling import loan_book
from application.modeling import borrowings

pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:,.2f}".format

## **Inputs**

In [4]:
start_date = "2023-01"
months_to_forecast = 12
imtt = 0.01

In [5]:
disbursement_parameters = pd.read_csv("./data_templates/disbursement_parameters.csv", index_col=0)
disbursement_parameters.columns = pd.period_range(start_date,periods=int(disbursement_parameters.columns[-1]), freq="M")
disbursement_parameters.head()

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
AGENT_COMMISSION,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
AGENT_CONTRIBUTION_PERCENT,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10
BUSINESS_ACQUISITION_PERCENT,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
CREDIT_OFFICER_COMMISSION,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
CREDIT_OFFICER_SALARY,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00


In [6]:
other_parameters = pd.read_csv("./data_templates/other_parameters.csv", index_col=0)
other_parameters.columns = pd.period_range(start_date,periods=int(other_parameters.columns[-1]), freq="M")
other_parameters.head()

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
NEW_INVENTORY,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
INVENTORY_USED,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_INTERGROUP_RECEIVABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_OTHER_PAYABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
NEW_OTHER_RECEIVABLES,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [7]:
details_of_assets = pd.read_csv("./data_templates/details_of_assets.csv")
details_of_assets.head()

,Asset ID,Acquisition Date,Life,Book Value,Net Value,Salvage Value,Method,Depreciation
0,FF-HRE-0024,01/09/2015,10,380.00,75.96,0,straight_line,NaN
1,FF-HRE-0025,01/09/2015,10,370.00,74.04,0,straight_line,NaN
2,FF-HRE-0026,01/09/2015,10,370.00,74.04,0,straight_line,NaN
3,FF-HRE-0027,01/09/2015,10,630.00,126.00,0,straight_line,NaN
4,FF-HRE-0028,01/09/2015,10,630.00,126.00,0,straight_line,NaN


In [8]:
details_of_long_term_borrowing = pd.read_csv("./data_templates/details_of_long_term_borrowing.csv")
details_of_long_term_borrowing.head()

,Effective Date,Institution,Nominal Amount,Interest Rate,Tenure,Repayment Frequency,Frequency,Method
0,31/12/2022,ZIMNAT LIFE ASSURANCE,376455,0.06,60,Monthly (P+I),12,reducing_balance
1,06/05/2022,MASAWARA,5613,0.06,60,Monthly (P+I),12,reducing_balance
2,31/12/2022,Grandre,160000,0.06,36,Quaterly (I+Bullet P),4,straight_line
3,31/12/2022,MINERVA,15000,0.08,36,Bullet (P+I),0,straight_line
4,10/09/2022,ZAM-CARGO,364000,0.05,36,Annually(I) Principal rollover,12,straight_line


In [9]:
details_of_short_term_borrowing = pd.read_csv("./data_templates/details_of_short_term_borrowing.csv")
details_of_short_term_borrowing.head()

,Effective Date,Institution,Nominal Amount,Interest Rate,Tenure,Repayment Frequency,Frequency,Method
0,24/04/2023,FCB,"120,383.84",0.18,3,Bullet (P+I),0,straight_line
1,02/03/2023,ZIMNAT LIFE ASSURANCE,"50,000.00",0.14,6,Monthly (P+I),12,reducing_balance
2,31/12/2022,Zimnat Life Assurance,"20,000.00",0.08,12,Annually(I) Principal rollover,1,straight_line
3,31/12/2022,Grandre,"100,000.00",0.12,12,Quaterly (I+Bullet P),4,straight_line
4,23/04/2023,FCB,"18,429.93",1.50,12,Bullet (P+I),0,straight_line


In [10]:
expenses_certain = pd.read_csv("./data_templates/expenses_certain.csv", index_col=0)
expenses_certain.head()

,Jan-23,Feb-23,Mar-23,Apr-23,May-23,Jun-23,Jul-23,Aug-23,Sep-23,Oct-23,Nov-23,Dec-23
Training,0.00,58.39,0.00,296.32,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
CILL,278.00,"1,313.00",277.00,"1,910.00","3,048.00","6,169.00",528.00,222.00,913.00,882.00,876.00,862.00
Bonus Provision,"4,224.11","3,673.14","3,128.97","2,484.77","1,408.04",603.44,640.02,761.47,"3,071.99","2,929.90","2,901.54","2,837.47"
Retrenchments,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
Staff Welfare,"1,081.56","3,861.43","3,645.71","3,203.79","4,191.68","2,658.42","19,268.45","16,790.41","16,910.35","17,031.13","17,152.78","17,275.30"


In [11]:
expenses_uncertain = pd.read_csv("./data_templates/expenses_uncertain.csv", index_col=0)
expenses_uncertain.head()

,1,2,3,4,5,6
Travel Costs,0.00,0.00,0.00,0.00,0.00,0.00
Entertainment,0.00,721.39,67.00,0.00,"4,530.00",124.00
Marketing Costs,"3,104.93","4,869.97","8,450.63","4,098.90","5,692.00",195.60
Group Marketing Costs,"7,145.92","7,543.70","12,847.68","9,277.20","6,097.62","15,314.38"
Rental Costs,910.55,"1,168.40",583.42,"1,479.51","1,601.02","1,315.55"


In [12]:
opening_balances = pd.read_csv("./data_templates/opening_balances.csv")
opening_balances

,TRADE_RECEIVABLES,TRADE_PAYABLES,LOAN_BOOK,CASH_ON_HAND,ISSUED_SHARE_CAPITAL,SHARE_PREMIUM,OTHER_COMPONENTS_OF_EQUITY,TREASURY_SHARES,RETAINED_EARNINGS,LONG_TERM_LOANS,SHORT_TERM_LOANS,INTERCOMPANY_LOANS,DEFERED_TAXATION,OTHER_RECEIVABLES,OTHER_PAYABLES,PROVISION_FOR_TAXATION,PROVISION_FOR_CREDIT_LOSS,INVESTMENT_IN_SUBSIDIARIES,INVESTMENT_IN_ASSOCIATES,INVESTMENT_PROPERTIES,EQUITY_INVESTMENTS,LONG_TERM_MONEY_MARKET_INVESTMENTS,LOANS_TO_RELATED_ENTITIES,TREASURY_SHARES.1,INTANGIBLE_ASSETS,INVENTORIES,OTHER_RECEIVABLES.1,INTERGROUP_RECEIVABLES,SHORT_TERM_MONEY_MARKET_INVESTMENTS
0,1092458,938595,2806038,331856,7050,110,59784,0,876511,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
existing_loans = pd.read_csv("./data_templates/existing_loans.csv")
existing_loans.head()

,Loan Number,Disbursement Date,Loan Term,Loan Amount,Closing Balance,Interest Rate,Admin Fee,Credit Insurance Fee,Loan Type
0,"100,120,644,000,001",13/10/2021,60,"20,000.00","-14,769.50",0.00,0.02,0.02,B2B
1,"100,120,644,000,002",15/10/2021,60,"12,000.00","-8,855.52",0.00,0.02,0.02,B2B
2,"100,120,644,000,003",25/10/2021,60,"20,000.00","-14,708.39",0.00,0.02,0.02,B2B
3,"100,120,644,000,004",26/10/2021,60,"20,000.00","-14,703.34",0.00,0.02,0.02,B2B
4,"100,120,644,000,005",28/10/2021,60,"20,000.00","-14,693.08",0.00,0.02,0.02,B2B


## **New Disbursements**

In [14]:
new_disbursements_df  = disbursements.calculate_new_disbursements(disbursement_parameters=disbursement_parameters)
new_disbursements_df

,b2b_disbursements,sme_disbursements,consumer_ssb_disbursements,consumer_pvt_disbursements,total
2023-01,"57,011.05","360,000.00","48,000.00","60,000.00","537,011.05"
2023-02,"27,480.25","432,000.00","60,000.00","75,000.00","609,480.25"
2023-03,"27,480.25","480,000.00","72,000.00","108,000.00","723,480.25"
2023-04,"34,900.00","360,000.00","80,000.00","120,000.00","634,900.00"
2023-05,"66,173.00","324,000.00","96,000.00","144,000.00","678,173.00"
2023-06,"18,709.25","378,000.00","96,000.00","162,000.00","720,709.25"
2023-07,"40,650.00","420,000.00","112,000.00","189,000.00","838,650.00"
2023-08,"8,680.00","595,000.00","120,000.00","202,500.00","1,008,680.00"
2023-09,"65,000.00","700,000.00","120,000.00","225,000.00","1,215,000.00"
2023-10,"70,000.00","990,000.00","120,000.00","225,000.00","1,510,000.00"


### **Calculate Repayment Amount For New Disbursements**

In [15]:
monthly_repayment_new_disbursements_df = (
    interest_income.calculate_monthly_repayments_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        disbursement_parameters=disbursement_parameters,
    )
)
monthly_repayment_new_disbursements_df.head()

,sme_monthly_repayment,b2b_monthly_repayment,consumer_ssb_monthly_repayment,consumer_pvt_monthly_repayment,total
2023-01,"81,451.18","5,779.79","8,006.34","10,007.93","105,245.24"
2023-02,"97,741.42","2,785.95","10,007.93","12,509.91","123,045.21"
2023-03,"108,601.58","2,785.95","12,009.51","18,014.27","141,411.31"
2023-04,"81,451.18","3,538.17","11,172.05","16,758.08","112,919.48"
2023-05,"73,306.06","6,708.63","13,406.46","20,109.69","113,530.85"


### **Generate Loan Schedules For New Disbursements**

In [16]:
loan_schedules_for_all_new_disbursements = (
    interest_income.generate_loan_schedules_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        disbursement_parameters=disbursement_parameters,
        monthly_repayment_new_disbursements_df=monthly_repayment_new_disbursements_df,
        months_to_forecast=months_to_forecast,
    )
)
loan_schedules_for_all_new_disbursements.keys()

dict_keys(['sme_loan_schedules', 'b2b_loan_schedules', 'consumer_ssb_loan_schedules', 'consumer_pvt_loan_schedules'])

In [17]:
loan_schedules_for_all_new_disbursements['sme_loan_schedules'].keys()

dict_keys(['interest', 'capital_repayments', 'outstanding_at_start'])

### **Capital Repayment New Disbursement**

In [18]:
capital_repayment_new_disbursements_df = interest_income.generate_capital_repayment_new_disbursements_df(
    loan_schedules_for_all_new_disbursements=loan_schedules_for_all_new_disbursements
)
capital_repayment_new_disbursements_df.head()

,sme_capital_repayments,b2b_capital_repayments,consumer_pvt_capital_repayments,consumer_ssb_capital_repayments,total
Feb-2023,"47,251.18","5,637.26","4,607.93","3,686.34","61,182.71"
Mar-2023,"108,441.46","8,368.61","10,782.55","8,626.04","136,218.66"
Apr-2023,"181,744.98","11,106.78","20,047.25","14,931.90","227,830.91"
May-2023,"246,261.93","14,585.46","27,809.58","20,247.82","308,904.80"
Jun-2023,"312,182.88","21,165.12","37,462.14","26,836.59","397,646.73"


### **Interest Income New Disbursements**


In [19]:
interest_income_new_disbursements_df = interest_income.generate_interest_income_new_disbursements_df(
    loan_schedules_for_all_new_disbursements=loan_schedules_for_all_new_disbursements
)

interest_income_new_disbursements_df

,sme_interest_income,b2b_interest_income,consumer_pvt_interest_income,consumer_ssb_interest_income,total
Feb-2023,"34,200.00",142.53,"5,400.00","4,320.00","44,062.53"
Mar-2023,"70,751.14",197.14,"11,735.29","9,388.23","92,071.79"
Apr-2023,"106,049.20",244.91,"20,484.86","15,091.89","141,870.86"
May-2023,"122,983.43",304.40,"29,480.60","20,948.01","173,716.44"
Jun-2023,"130,368.54",433.37,"39,937.74","27,765.71","198,505.36"
Jul-2023,"136,621.17",427.23,"51,146.15","33,990.42","222,184.96"
Aug-2023,"139,333.04",471.18,"63,757.21","41,008.77","244,570.19"
Sep-2023,"157,637.89",425.02,"76,342.80","47,971.09","282,376.80"
Oct-2023,"184,153.18",517.34,"89,540.97","54,220.18","328,431.67"
Nov-2023,"236,008.72",605.92,"101,099.04","59,691.86","397,405.54"


### **Calculate Admin Fee On New Disbursements**

In [20]:
admin_fee_for_all_new_disbursements_df = other_income.calculate_admin_fee_for_all_new_disbursements(
    new_disbursements_df=new_disbursements_df,
    disbursement_parameters=disbursement_parameters,
    months_to_forecast=months_to_forecast,
)

admin_fee_for_all_new_disbursements_df

,sme_admin_fee,b2b_admin_fee,consumer_ssb_admin_fee,consumer_pvt_admin_fee,total
Jan-2023,"4,098.00",114.02,252.18,372.55,"4,836.75"
Feb-2023,"9,015.60",168.98,567.42,838.23,"10,590.23"
Mar-2023,"14,479.60",223.94,945.69,"1,508.81","17,158.04"
Apr-2023,"18,577.60",293.74,"1,366.00","2,253.90","22,491.24"
May-2023,"22,265.80",426.09,"1,870.37","3,148.01","27,710.27"
Jun-2023,"26,568.70",463.51,"2,374.74","4,153.88","33,560.83"
Jul-2023,"27,251.70",544.81,"2,963.17","5,327.40","36,087.08"
Aug-2023,"29,107.18",562.17,"3,593.63","6,584.74","39,847.72"
Sep-2023,"31,611.52",692.17,"4,224.09","7,981.79","44,509.56"
Oct-2023,"38,783.02",832.17,"4,854.55","9,378.83","53,848.57"


### **Calculate Credit Life Insurance On New Disbursement**

In [23]:
credit_insurance_fee_for_all_new_disbursements_df = other_income.calculate_credit_insurance_fee_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        disbursement_parameters=disbursement_parameters,
        months_to_forecast=months_to_forecast,
    )

credit_insurance_fee_for_all_new_disbursements_df

,sme_credit_insurance_fee,b2b_credit_insurance_fee,consumer_ssb_credit_insurance_fee,consumer_pvt_credit_insurance_fee,total
Jan-2023,"1,020.00",96.92,62.77,92.73,"1,272.42"
Feb-2023,"2,244.00",143.64,141.23,208.64,"2,737.50"
Mar-2023,"3,604.00",190.35,235.38,375.55,"4,405.28"
Apr-2023,"4,624.00",249.68,340.00,561.00,"5,774.68"
May-2023,"5,542.00",362.18,465.54,783.55,"7,153.26"
Jun-2023,"6,613.00",393.98,591.08,"1,033.91","8,631.97"
Jul-2023,"6,783.00",463.09,737.54,"1,326.00","9,309.62"
Aug-2023,"7,244.83",477.84,894.46,"1,638.95","10,256.09"
Sep-2023,"7,868.17",588.34,"1,051.38","1,986.68","11,494.58"
Oct-2023,"9,653.17",707.34,"1,208.31","2,334.41","13,903.23"


### **Calculate For Credit Loss For New Disbursements**

In [24]:
provision_for_credit_loss_for_all_new_disbursements = (
    expenses.calculate_provision_for_credit_loss_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df, disbursement_parameters=disbursement_parameters
    )
)

provision_for_credit_loss_for_all_new_disbursements

,sme_provision_for_credit_loss,b2b_provision_for_credit_loss,consumer_ssb_provision_for_credit_loss,consumer_pvt_provision_for_credit_loss,total
Jan-2023,"7,200.00",0.00,960.00,"1,800.00","9,960.00"
Feb-2023,"8,640.00",0.00,"1,200.00","2,250.00","12,090.00"
Mar-2023,"9,600.00",0.00,"1,440.00","3,240.00","14,280.00"
Apr-2023,"7,200.00",0.00,"1,600.00","3,600.00","12,400.00"
May-2023,"6,480.00",0.00,"1,920.00","4,320.00","12,720.00"
Jun-2023,"7,560.00",0.00,"1,920.00","4,860.00","14,340.00"
Jul-2023,"8,400.00",0.00,"2,240.00","5,670.00","16,310.00"
Aug-2023,"11,900.00",0.00,"2,400.00","6,075.00","20,375.00"
Sep-2023,"14,000.00",0.00,"2,400.00","6,750.00","23,150.00"
Oct-2023,"19,800.00",0.00,"2,400.00","6,750.00","28,950.00"


## **Existing Loans**

### **Generate Loan Schedules For Existing Loans**


In [25]:
existing_loans = helper.columns_to_snake_case(existing_loans)
existing_loans = existing_loans.loc[existing_loans["closing_balance"] > 0]

In [26]:
existing_loans_schedules = borrowings.calculate_reducing_balance_loans_schedules(
    interest_rates=existing_loans["interest_rate"],
    effective_dates=existing_loans["disbursement_date"],
    frequencies=(existing_loans['interest_rate']*0+12),
    loan_identifiers=existing_loans["loan_number"],
    tenures=existing_loans["loan_term"],
    amounts=existing_loans["loan_amount"],
    is_interest_rate_annual=False
)

existing_loans_schedules.keys()

dict_keys(['outstanding_balance_at_start', 'capital_repayments', 'interest_payments', 'repayments'])

In [27]:
existing_loans_schedules['outstanding_balance_at_start']

,Jan-2019,Feb-2019,Mar-2019,Apr-2019,May-2019,Jun-2019,Jul-2019,Aug-2019,Sep-2019,Oct-2019,Nov-2019,Dec-2019,Jan-2020,Feb-2020,Mar-2020,Apr-2020,May-2020,Jun-2020,Jul-2020,Aug-2020,Sep-2020,Oct-2020,Nov-2020,Dec-2020,Jan-2021,Feb-2021,Mar-2021,Apr-2021,May-2021,Jun-2021,Jul-2021,Aug-2021,Sep-2021,Oct-2021,Nov-2021,Dec-2021,Jan-2022,Feb-2022,Mar-2022,Apr-2022,May-2022,Jun-2022,Jul-2022,Aug-2022,Sep-2022,Oct-2022,Nov-2022,Dec-2022,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025
"100,120,692,000,003",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"20,000.00","19,468.38","18,935.42","18,401.14","17,865.52","17,328.56","16,790.25","16,250.60","15,709.61","15,167.26","14,623.55","14,078.48","13,532.06","12,984.26","12,435.10","11,884.56","11,332.65","10,779.36","10,224.68","9,668.62","9,111.17","8,552.32","7,992.08","7,430.43","6,867.38","6,302.93","5,737.06","5,169.78","4,601.08","4,030.96","3,459.41","2,886.44","2,312.03","1,736.18","1,158.90",580.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,005",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"20,000.00","19,468.38","18,935.42","18,401.14","17,865.52","17,328.56","16,790.25","16,250.60","15,709.61","15,167.26","14,623.55","14,078.48","13,532.06","12,984.26","12,435.10","11,884.56","11,332.65","10,779.36","10,224.68","9,668.62","9,111.17","8,552.32","7,992.08","7,430.43","6,867.38","6,302.93","5,737.06","5,169.78","4,601.08","4,030.96","3,459.41","2,886.44","2,312.03","1,736.18","1,158.90",580.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,007",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"20,000.00","19,468.38","18,935.42","18,401.14","17,865.52","17,328.56","16,790.25","16,250.60","15,709.61","15,167.26","14,623.55","14,078.48","13,532.06","12,984.26","12,435.10","11,884.56","11,332.65","10,779.36","10,224.68","9,668.62","9,111.17","8,552.32","7,992.08","7,430.43","6,867.38","6,302.93","5,737.06","5,169.78","4,601.08","4,030.96","3,459.41","2,886.44","2,312.03","1,736.18","1,158.90",580.17,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,011",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"19,352.00","18,837.60","18,321.91","17,804.94","17,286.67","16,767.11","16,246.25","15,724.08","15,200.61","14,675.84","14,149.75","13,622.34","13,093.62","12,563.57","12,032.20","11,499.50","10,965.47","10,430.11","9,893.40","9,355.36","8,815.96","8,275.22","7,733.13","7,189.69","6,644.88","6,098.71","5,551.18","5,002.28","4,452.01","3,900.36","3,347.33","2,792.92","2,237.12","1,679.93","1,121.35",561.38
"100,120,692,000,016",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,800.00,667.50,534.66,401.50,268.00,134.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [28]:
existing_loans_schedules['interest_payments']

,Jan-2019,Feb-2019,Mar-2019,Apr-2019,May-2019,Jun-2019,Jul-2019,Aug-2019,Sep-2019,Oct-2019,Nov-2019,Dec-2019,Jan-2020,Feb-2020,Mar-2020,Apr-2020,May-2020,Jun-2020,Jul-2020,Aug-2020,Sep-2020,Oct-2020,Nov-2020,Dec-2020,Jan-2021,Feb-2021,Mar-2021,Apr-2021,May-2021,Jun-2021,Jul-2021,Aug-2021,Sep-2021,Oct-2021,Nov-2021,Dec-2021,Jan-2022,Feb-2022,Mar-2022,Apr-2022,May-2022,Jun-2022,Jul-2022,Aug-2022,Sep-2022,Oct-2022,Nov-2022,Dec-2022,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025
"100,120,692,000,003",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,50.00,48.67,47.34,46.00,44.66,43.32,41.98,40.63,39.27,37.92,36.56,35.20,33.83,32.46,31.09,29.71,28.33,26.95,25.56,24.17,22.78,21.38,19.98,18.58,17.17,15.76,14.34,12.92,11.50,10.08,8.65,7.22,5.78,4.34,2.90,1.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,005",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,50.00,48.67,47.34,46.00,44.66,43.32,41.98,40.63,39.27,37.92,36.56,35.20,33.83,32.46,31.09,29.71,28.33,26.95,25.56,24.17,22.78,21.38,19.98,18.58,17.17,15.76,14.34,12.92,11.50,10.08,8.65,7.22,5.78,4.34,2.90,1.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,007",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,50.00,48.67,47.34,46.00,44.66,43.32,41.98,40.63,39.27,37.92,36.56,35.20,33.83,32.46,31.09,29.71,28.33,26.95,25.56,24.17,22.78,21.38,19.98,18.58,17.17,15.76,14.34,12.92,11.50,10.08,8.65,7.22,5.78,4.34,2.90,1.45,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,011",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,48.38,47.09,45.80,44.51,43.22,41.92,40.62,39.31,38.00,36.69,35.37,34.06,32.73,31.41,30.08,28.75,27.41,26.08,24.73,23.39,22.04,20.69,19.33,17.97,16.61,15.25,13.88,12.51,11.13,9.75,8.37,6.98,5.59,4.20,2.80,1.40
"100,120,692,000,016",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,1.67,1.34,1.00,0.67,0.34,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"100,120,708,000,643",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,26.30,24.28,22.08,19.68,17.07,14.21,11.11,7.72,4.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,708,000,697",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.12,8.69,6.04,3.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,

In [29]:
existing_loans_schedules['capital_repayments']

,Jan-2019,Feb-2019,Mar-2019,Apr-2019,May-2019,Jun-2019,Jul-2019,Aug-2019,Sep-2019,Oct-2019,Nov-2019,Dec-2019,Jan-2020,Feb-2020,Mar-2020,Apr-2020,May-2020,Jun-2020,Jul-2020,Aug-2020,Sep-2020,Oct-2020,Nov-2020,Dec-2020,Jan-2021,Feb-2021,Mar-2021,Apr-2021,May-2021,Jun-2021,Jul-2021,Aug-2021,Sep-2021,Oct-2021,Nov-2021,Dec-2021,Jan-2022,Feb-2022,Mar-2022,Apr-2022,May-2022,Jun-2022,Jul-2022,Aug-2022,Sep-2022,Oct-2022,Nov-2022,Dec-2022,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025
"100,120,692,000,003",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,531.62,532.95,534.29,535.62,536.96,538.30,539.65,541.00,542.35,543.71,545.07,546.43,547.79,549.16,550.54,551.91,553.29,554.68,556.06,557.45,558.85,560.24,561.64,563.05,564.46,565.87,567.28,568.70,570.12,571.55,572.98,574.41,575.84,577.28,578.73,580.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,005",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,531.62,532.95,534.29,535.62,536.96,538.30,539.65,541.00,542.35,543.71,545.07,546.43,547.79,549.16,550.54,551.91,553.29,554.68,556.06,557.45,558.85,560.24,561.64,563.05,564.46,565.87,567.28,568.70,570.12,571.55,572.98,574.41,575.84,577.28,578.73,580.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,007",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,531.62,532.95,534.29,535.62,536.96,538.30,539.65,541.00,542.35,543.71,545.07,546.43,547.79,549.16,550.54,551.91,553.29,554.68,556.06,557.45,558.85,560.24,561.64,563.05,564.46,565.87,567.28,568.70,570.12,571.55,572.98,574.41,575.84,577.28,578.73,580.17,0.00,0.00,0.00,0.00,0.00
"100,120,692,000,011",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,514.40,515.69,516.97,518.27,519.56,520.86,522.16,523.47,524.78,526.09,527.41,528.72,530.05,531.37,532.70,534.03,535.37,536.70,538.05,539.39,540.74,542.09,543.45,544.81,546.17,547.53,548.90,550.27,551.65,553.03,554.41,555.80,557.19,558.58,559.98,561.38
"100,120,692,000,016",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,132.50,132.83,133.17,133.50,133.83,134.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"100,120,708,000,643",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,22.45,24.47,26.67,29.07,31.68,34.54,37.64,41.03,44.73,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
"100,120,708,000,697",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0

### **Capital Repayment for Existing Loans**

In [30]:
capital_repayment_existing_loans = existing_loans_schedules['capital_repayments'].sum()
capital_repayment_existing_loans.loc[helper.generate_columns(start_date,months_to_forecast)]

Jan-2023   147,567.56
Feb-2023   144,585.73
Mar-2023   160,532.02
Apr-2023   172,443.92
May-2023   109,934.28
Jun-2023   100,375.41
Jul-2023    63,408.12
Aug-2023    59,655.62
Sep-2023    10,092.45
Oct-2023     6,747.14
Nov-2023     4,764.47
Dec-2023     2,982.53
dtype: float64

### **Intererest Income For Existing Loans**

In [31]:
interest_income_existing_loans = existing_loans_schedules['interest_payments'].sum()
interest_income_existing_loans.loc[helper.generate_columns(start_date,months_to_forecast)]

Jan-2023   49,585.60
Feb-2023   45,495.46
Mar-2023   50,220.30
Apr-2023   38,392.91
May-2023   29,702.43
Jun-2023   20,046.99
Jul-2023   11,142.26
Aug-2023    6,626.19
Sep-2023    1,283.41
Oct-2023      637.27
Nov-2023      296.86
Dec-2023      143.96
dtype: float64

### **Calculate Other Income Existing Loans**

In [32]:
existing_loans = helper.columns_to_snake_case(existing_loans)
existing_loans = existing_loans.loc[existing_loans["closing_balance"] > 0]

In [33]:
other_income_existing_loans_df = other_income.calculate_other_income_existing_loans(
    existing_loans=existing_loans,
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)
other_income_existing_loans_df

c:\Users\HP\Desktop\Claxon\Budgeting\manage-tenants-template\application\modeling\borrowings.py:53: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  pd.date_range(


,credit_insurance_fee_existing_loans,admin_fee_existing_loans,total
Jan-2023,"2,071.79","7,853.24","9,925.03"
Feb-2023,"1,790.03","6,932.88","8,722.91"
Mar-2023,"2,086.38","8,127.91","10,214.29"
Apr-2023,"2,335.76","7,886.12","10,221.88"
May-2023,"1,516.71","5,854.88","7,371.59"
Jun-2023,"1,347.63","5,149.53","6,497.17"
Jul-2023,"1,037.57","3,959.76","4,997.33"
Aug-2023,917.71,"3,484.26","4,401.97"
Sep-2023,542.77,"1,976.64","2,519.41"
Oct-2023,695.72,"2,594.76","3,290.48"


## **Interest Income**

In [34]:
total_interest_income = interest_income.aggregate_new_and_existing_loans_interest_income(
    interest_income_new_disbursements_df=interest_income_new_disbursements_df,
    interest_income_existing_loans= interest_income_existing_loans,
    start_date= start_date, 
    months_to_forecast= months_to_forecast
)
total_interest_income

Jan-2023    49,585.60
Feb-2023    89,557.99
Mar-2023   142,292.09
Apr-2023   180,263.76
May-2023   203,418.87
Jun-2023   218,552.35
Jul-2023   233,327.22
Aug-2023   251,196.38
Sep-2023   283,660.21
Oct-2023   329,068.93
Nov-2023   397,702.40
Dec-2023   469,658.41
dtype: float64

## **Other Income**

In [35]:
other_income_df = other_income.aggregate_other_income(
    admin_fee_for_all_new_disbursements_df=admin_fee_for_all_new_disbursements_df,
    admin_fee_existing_loans=other_income_existing_loans_df['admin_fee_existing_loans'],
    credit_insurance_fee_existing_loans=other_income_existing_loans_df['credit_insurance_fee_existing_loans'],
    credit_insurance_fee_for_all_new_disbursements_df=credit_insurance_fee_for_all_new_disbursements, 
    start_date=start_date, 
    months_to_forecast=months_to_forecast
)
other_income_df

,admin_fee,credit_insurance_fee,total
Jan-2023,"17,526.75","4,616.62","22,143.37"
Feb-2023,"28,113.33","7,265.04","35,378.37"
Mar-2023,"42,444.00","10,896.94","53,340.94"
Apr-2023,"52,868.61","13,885.13","66,753.73"
May-2023,"61,275.42","15,823.23","77,098.65"
Jun-2023,"72,271.19","18,611.57","90,882.76"
Jul-2023,"76,133.91","19,656.82","95,790.73"
Aug-2023,"83,179.70","21,429.90","104,609.60"
Sep-2023,"90,995.77","23,531.92","114,527.68"
Oct-2023,"110,291.90","28,502.17","138,794.07"


## **Depreciation**

In [36]:
details_of_assets = helper.columns_to_snake_case(details_of_assets)
details_of_assets.head()

,asset_id,acquisition_date,life,book_value,net_value,salvage_value,method,depreciation
0,FF-HRE-0024,01/09/2015,10,380.00,75.96,0,straight_line,NaN
1,FF-HRE-0025,01/09/2015,10,370.00,74.04,0,straight_line,NaN
2,FF-HRE-0026,01/09/2015,10,370.00,74.04,0,straight_line,NaN
3,FF-HRE-0027,01/09/2015,10,630.00,126.00,0,straight_line,NaN
4,FF-HRE-0028,01/09/2015,10,630.00,126.00,0,straight_line,NaN


In [37]:
depreciations_and_nbvs = depreciation.calculate_depreciations_and_nbvs(
    details_of_assets=details_of_assets,
    start_date=start_date,
    months_to_forecast=months_to_forecast
)
depreciations_and_nbvs.keys()

dict_keys(['nbvs', 'dpns'])

In [38]:
depreciations_and_nbvs['nbvs']

,CE-HRE-0039,CE-HRE-0040,CE-HRE-0041,CE-HRE-0042,CE-HRE-0043,CE-HRE-0044,CE-HRE-0045,CE-HRE-0046,CE-HRE-0047,CE-HRE-0048,CE-HRE-0049,CE-HRE-0050,CE-HRE-0051,CE-HRE-0052,CE-HRE-0053,CE-HRE-0070,CE-HRE-0071,CE-HRE-0075,CE-HRE-0077,CE-HRE-0081,CE-HRE-0085,CE-HRE-0086,CE-HRE-0087,CE-HRE-0089,CE-HRE-0090,CE-HRE-0091,CE-HRE-0092,CE-HRE-0093,CE-HRE-0094,CE-HRE-0095,CE-HRE-0096,CE-HRE-0097,CE-HRE-0098,CE-HRE-0100,CE-HRE-0101,CE-HRE-0102,CE-HRE-0103,CE-HRE-0104,CE-HRE-0105,CE-HRE-0106,CE-HRE-0107,CE-HRE-0108,CE-HRE-0109,CE-HRE-0110,CE-HRE-0111,CE-HRE-0112,CE-ZFX-0003,CE-ZFX-0004,CE-ZFX-0006,CE-ZFX-0007,CS-HRE-0003,CS-HRE-0004,CS-HRE-0005,CS-ZFX-0002,EL-BYO-0036,EL-HRE-0007,EL-HRE-0017,EL-HRE-0032,EL-HRE-0037,EL-HRE-0038,FF-BYO-0057,FF-BYO-0058,FF-BYO-0059,FF-BYO-0060,FF-BYO-0061,FF-BYO-0062,FF-BYO-0063,FF-BYO-0064,FF-BYO-0065,FF-BYO-0066,FF-BYO-0067,FF-BYO-0068,FF-BYO-0069,FF-BYO-0070,FF-BYO-0075,FF-BYO-0146,FF-HRE-0001,FF-HRE-0002,FF-HRE-0003,FF-HRE-0004,FF-HRE-0005,FF-HRE-0006,FF-HRE-0007,FF-HRE-0008,FF-HRE-0009,FF-HRE-0010,FF-HRE-0011,FF-HRE-0012,FF-HRE-0013,FF-HRE-0014,FF-HRE-0015,FF-HRE-0016,FF-HRE-0017,FF-HRE-0018,FF-HRE-0019,FF-HRE-0020,FF-HRE-0021,FF-HRE-0022,FF-HRE-0023,FF-HRE-0024,FF-HRE-0025,FF-HRE-0026,FF-HRE-0027,FF-HRE-0028,FF-HRE-0029,FF-HRE-0030,FF-HRE-0031,FF-HRE-0032,FF-HRE-0033,FF-HRE-0034,FF-HRE-0035,FF-HRE-0036,FF-HRE-0037,FF-HRE-0041,FF-HRE-0042,FF-HRE-0043,FF-HRE-0045,FF-HRE-0050,FF-HRE-0051,FF-HRE-0052,FF-HRE-0054,FF-HRE-0055,FF-HRE-0071,FF-HRE-0072,FF-HRE-0073,FF-HRE-0074,FF-HRE-0076,FF-HRE-0086,FF-HRE-0087,FF-HRE-0088,FF-HRE-0089,FF-HRE-0090,FF-HRE-0091,FF-HRE-0092,FF-HRE-0093,FF-HRE-0094,FF-HRE-0095,FF-HRE-0096,FF-HRE-0097,FF-HRE-0098,FF-HRE-0099,FF-HRE-0100,FF-HRE-0101,FF-HRE-0102,FF-HRE-0103,FF-HRE-0104,FF-HRE-0105,FF-HRE-0107,FF-HRE-0108,FF-HRE-0109,FF-HRE-0110,FF-HRE-0111,FF-HRE-0112,FF-HRE-0113,FF-HRE-0114,FF-HRE-0115,FF-HRE-0116,FF-HRE-0117,FF-HRE-0118,FF-HRE-0119,FF-HRE-0120,FF-HRE-0121,FF-HRE-0122,FF-HRE-0123,FF-HRE-0124,FF-HRE-0125,FF-HRE-0126,FF-HRE-0127,FF-HRE-0128,FF-HRE-0129,FF-HRE-0130,FF-HRE-0131,FF-HRE-0132,FF-HRE-0133,FF-HRE-0137,FF-HRE-0138,FF-HRE-0140,FF-HRE-0141,FF-HRE-0143,FF-HRE-0144,FF-HRE-0145,FF-HRE-0147,FF-HRE-0148,FF-HRE-0149,FF-HRE-0150,FF-HRE-0151,FF-HRE-0152,FF-HRE-0153,FF-HRE-0154,FF-HRE-0155,FF-HRE-0156,FF-HRE-0157,FF-HRE-0158,FF-HRE-0159,FF-HRE-0160,FF-HRE-0161,FF-HRE-0162,FF-HRE-0163,FF-HRE-0164,FF-HRE-0165,FF-HRE-0166,FF-HRE-0167,FF-HRE-0168,FF-HRE-0169,FF-HRE-0170,FF-HRE-0171,FF-HRE-0172,FF-HRE-0173,FF-HRE-0174,FF-HRE-0175,FF-HRE-0176,FF-HRE-0177,FF-HRE-0178,FF-HRE-0179,FF-HRE-0180,FF-HRE-0181,FF-HRE-0182,FF-HRE-0183,FF-HRE-0184,FF-HRE-0186,FF-HRE-0187,FF-HRE-0188,FF-HRE-0189,FF-HRE-0190,FF-HRE-0191,FF-HRE-0192,FF-HRE-0193,FF-HRE-0194,FF-HRE-0195,FF-HRE-0196,FF-HRE-0198,FF-HRE-0199,FF-HRE-0201,FF-HRE-0202,FF-HRE-0203,FF-HRE-0204,FF-ZFX-0001,FF-ZFX-0002,FF-ZFX-0003,FF-ZFX-0004,FF-ZFX-0007,MV-HRE-0003,MV-HRE-0006,MV-HRE-0008,MV-HRE-0009,total
Jan-2023,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,106.57,44.09,"1,401.87","1,606.82","3,861.00","10,191.43","6,200.10","32,025.00","11,215.16","139,727.90","101,695.40","107,055.17","148,353.15","148,353.15","148,353.15","148,353.15","148,353.14","148,353.14","148,353.14","1,679,763.48","3,529,883.10","891,919.58","891,919.58","891,919.58","891,919.58","891,919.58","891,919.58","891,919.58","891,919.58","1,337,675.23","1,016,693.88","10,129,620.41","2,177.06","2,177.06",635.46,"2,518.75","1,026.55",840.00,"2,287.07","11,586.96","12,303,570.33",71.15,168.34,"1,419.49","99,600.17","12,303,570.33",21.86,22.56,21.86,97.81,46.03,46.03,46.03,46.03,46.03,46.03,46.03,46.03,46.03,47.90,607.75,640.50,72.16,72.16,72.16,54.17,54.17,54.17,54.17,54.17,54.17,54.17,54.17,34.16,34.16,34.16,34.16,34.16,34.16,30.31,30.31,30.32,30.31,46.34,46.34,72.79,70.96,70.96,120.75,120.75,120.75,120.75,21.04,"1,082.39",529.82,392.04,462.00,567.95,96.00,390.00,66.00,266.04,183.95,"1,993.42",127.78,127.78,40.10,40.

In [39]:
depreciations_and_nbvs['dpns']

,CE-HRE-0039,CE-HRE-0040,CE-HRE-0041,CE-HRE-0042,CE-HRE-0043,CE-HRE-0044,CE-HRE-0045,CE-HRE-0046,CE-HRE-0047,CE-HRE-0048,CE-HRE-0049,CE-HRE-0050,CE-HRE-0051,CE-HRE-0052,CE-HRE-0053,CE-HRE-0070,CE-HRE-0071,CE-HRE-0075,CE-HRE-0077,CE-HRE-0081,CE-HRE-0085,CE-HRE-0086,CE-HRE-0087,CE-HRE-0089,CE-HRE-0090,CE-HRE-0091,CE-HRE-0092,CE-HRE-0093,CE-HRE-0094,CE-HRE-0095,CE-HRE-0096,CE-HRE-0097,CE-HRE-0098,CE-HRE-0100,CE-HRE-0101,CE-HRE-0102,CE-HRE-0103,CE-HRE-0104,CE-HRE-0105,CE-HRE-0106,CE-HRE-0107,CE-HRE-0108,CE-HRE-0109,CE-HRE-0110,CE-HRE-0111,CE-HRE-0112,CE-ZFX-0003,CE-ZFX-0004,CE-ZFX-0006,CE-ZFX-0007,CS-HRE-0003,CS-HRE-0004,CS-HRE-0005,CS-ZFX-0002,EL-BYO-0036,EL-HRE-0007,EL-HRE-0017,EL-HRE-0032,EL-HRE-0037,EL-HRE-0038,FF-BYO-0057,FF-BYO-0058,FF-BYO-0059,FF-BYO-0060,FF-BYO-0061,FF-BYO-0062,FF-BYO-0063,FF-BYO-0064,FF-BYO-0065,FF-BYO-0066,FF-BYO-0067,FF-BYO-0068,FF-BYO-0069,FF-BYO-0070,FF-BYO-0075,FF-BYO-0146,FF-HRE-0001,FF-HRE-0002,FF-HRE-0003,FF-HRE-0004,FF-HRE-0005,FF-HRE-0006,FF-HRE-0007,FF-HRE-0008,FF-HRE-0009,FF-HRE-0010,FF-HRE-0011,FF-HRE-0012,FF-HRE-0013,FF-HRE-0014,FF-HRE-0015,FF-HRE-0016,FF-HRE-0017,FF-HRE-0018,FF-HRE-0019,FF-HRE-0020,FF-HRE-0021,FF-HRE-0022,FF-HRE-0023,FF-HRE-0024,FF-HRE-0025,FF-HRE-0026,FF-HRE-0027,FF-HRE-0028,FF-HRE-0029,FF-HRE-0030,FF-HRE-0031,FF-HRE-0032,FF-HRE-0033,FF-HRE-0034,FF-HRE-0035,FF-HRE-0036,FF-HRE-0037,FF-HRE-0041,FF-HRE-0042,FF-HRE-0043,FF-HRE-0045,FF-HRE-0050,FF-HRE-0051,FF-HRE-0052,FF-HRE-0054,FF-HRE-0055,FF-HRE-0071,FF-HRE-0072,FF-HRE-0073,FF-HRE-0074,FF-HRE-0076,FF-HRE-0086,FF-HRE-0087,FF-HRE-0088,FF-HRE-0089,FF-HRE-0090,FF-HRE-0091,FF-HRE-0092,FF-HRE-0093,FF-HRE-0094,FF-HRE-0095,FF-HRE-0096,FF-HRE-0097,FF-HRE-0098,FF-HRE-0099,FF-HRE-0100,FF-HRE-0101,FF-HRE-0102,FF-HRE-0103,FF-HRE-0104,FF-HRE-0105,FF-HRE-0107,FF-HRE-0108,FF-HRE-0109,FF-HRE-0110,FF-HRE-0111,FF-HRE-0112,FF-HRE-0113,FF-HRE-0114,FF-HRE-0115,FF-HRE-0116,FF-HRE-0117,FF-HRE-0118,FF-HRE-0119,FF-HRE-0120,FF-HRE-0121,FF-HRE-0122,FF-HRE-0123,FF-HRE-0124,FF-HRE-0125,FF-HRE-0126,FF-HRE-0127,FF-HRE-0128,FF-HRE-0129,FF-HRE-0130,FF-HRE-0131,FF-HRE-0132,FF-HRE-0133,FF-HRE-0137,FF-HRE-0138,FF-HRE-0140,FF-HRE-0141,FF-HRE-0143,FF-HRE-0144,FF-HRE-0145,FF-HRE-0147,FF-HRE-0148,FF-HRE-0149,FF-HRE-0150,FF-HRE-0151,FF-HRE-0152,FF-HRE-0153,FF-HRE-0154,FF-HRE-0155,FF-HRE-0156,FF-HRE-0157,FF-HRE-0158,FF-HRE-0159,FF-HRE-0160,FF-HRE-0161,FF-HRE-0162,FF-HRE-0163,FF-HRE-0164,FF-HRE-0165,FF-HRE-0166,FF-HRE-0167,FF-HRE-0168,FF-HRE-0169,FF-HRE-0170,FF-HRE-0171,FF-HRE-0172,FF-HRE-0173,FF-HRE-0174,FF-HRE-0175,FF-HRE-0176,FF-HRE-0177,FF-HRE-0178,FF-HRE-0179,FF-HRE-0180,FF-HRE-0181,FF-HRE-0182,FF-HRE-0183,FF-HRE-0184,FF-HRE-0186,FF-HRE-0187,FF-HRE-0188,FF-HRE-0189,FF-HRE-0190,FF-HRE-0191,FF-HRE-0192,FF-HRE-0193,FF-HRE-0194,FF-HRE-0195,FF-HRE-0196,FF-HRE-0198,FF-HRE-0199,FF-HRE-0201,FF-HRE-0202,FF-HRE-0203,FF-HRE-0204,FF-ZFX-0001,FF-ZFX-0002,FF-ZFX-0003,FF-ZFX-0004,FF-ZFX-0007,MV-HRE-0003,MV-HRE-0006,MV-HRE-0008,MV-HRE-0009,total
Jan-2023,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,2.18,22.04,477.91,267.81,429.00,636.97,364.71,"1,601.25",659.71,"3,493.20","2,542.39","2,611.10","3,532.22","3,532.22","3,532.22","3,532.22","3,532.22","3,532.22","3,532.22","32,303.12","63,033.63","15,927.14","15,927.14","15,927.14","15,927.14","15,927.14","15,927.14","15,927.14","15,927.14","26,228.93","19,551.81","177,712.01",197.92,197.92,52.95,193.75,342.19,60.00,762.35,827.64,"361,869.23",2.63,3.92,19.99,"3,434.49","361,869.23",0.70,0.70,0.70,3.15,1.49,1.49,1.49,1.49,1.49,1.49,1.49,1.49,1.49,1.54,19.61,10.50,3.14,3.14,3.14,2.35,2.35,2.35,2.35,2.35,2.35,2.35,2.35,1.48,1.48,1.48,1.48,1.48,1.48,1.32,1.32,1.32,1.32,2.02,2.02,3.17,3.08,3.08,5.25,5.25,5.25,5.25,0.92,45.98,22.08,16.33,19.25,23.67,4.00,16.25,2.75,11.08,7.67,77.34,4.92,4.92,1.48,1.48,2.49,2.04,1.00,1.00,13.04,8.79,22.83,1.91,3.81,3.66,2.53,104.55,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,1.91,3.66,6.17,5.62,4.48,1.75,4.83,1.50,13.00,5.96,8.67,30.33,5.20,5.96,8.67,34.67,27.08,13.

## **Salaries**

In [41]:
salaries_and_pension_and_statutory_contributions_df = (
    expenses.calculate_salaries_and_pension_and_statutory_contributions(
        new_disbursements_df=new_disbursements_df,
        disbursement_parameters=disbursement_parameters,
        other_parameters = other_parameters, 
        months_to_forecast=months_to_forecast,
        start_date=start_date,
    )
)
salaries_and_pension_and_statutory_contributions_df

,agent_commission,credit_officer_salaries,credit_officer_commission,other_staff_salaries,total,pensions_and_statutory_contributions
Jan-2023,"5,400.00",600.00,"3,600.00","17,000.00","26,600.00",NaN
Feb-2023,"6,750.00",600.00,"4,320.00","17,000.00","28,670.00",NaN
Mar-2023,"9,000.00",600.00,"4,800.00","17,000.00","31,400.00",NaN
Apr-2023,"10,000.00",600.00,"3,600.00","17,000.00","31,200.00",NaN
May-2023,"12,000.00",900.00,"3,240.00","17,000.00","33,140.00",NaN
Jun-2023,"12,900.00",900.00,"3,780.00","17,000.00","34,580.00",NaN
Jul-2023,"15,050.00","1,200.00","4,200.00","17,000.00","37,450.00",NaN
Aug-2023,"16,125.00","1,400.00","5,950.00","17,000.00","40,475.00",NaN
Sep-2023,"17,250.00","1,400.00","7,000.00","17,000.00","42,650.00",NaN
Oct-2023,"17,250.00","1,800.00","9,900.00","17,000.00","45,950.00",NaN


## **Provisions**


In [42]:
provision_for_credit_loss_for_all_new_disbursements_df = (
    expenses.calculate_provision_for_credit_loss_for_all_new_disbursements(
        new_disbursements_df=new_disbursements_df,
        disbursement_parameters=disbursement_parameters,
    )
)
provision_for_credit_loss_for_all_new_disbursements_df

,sme_provision_for_credit_loss,b2b_provision_for_credit_loss,consumer_ssb_provision_for_credit_loss,consumer_pvt_provision_for_credit_loss,total
Jan-2023,"7,200.00",0.00,960.00,"1,800.00","9,960.00"
Feb-2023,"8,640.00",0.00,"1,200.00","2,250.00","12,090.00"
Mar-2023,"9,600.00",0.00,"1,440.00","3,240.00","14,280.00"
Apr-2023,"7,200.00",0.00,"1,600.00","3,600.00","12,400.00"
May-2023,"6,480.00",0.00,"1,920.00","4,320.00","12,720.00"
Jun-2023,"7,560.00",0.00,"1,920.00","4,860.00","14,340.00"
Jul-2023,"8,400.00",0.00,"2,240.00","5,670.00","16,310.00"
Aug-2023,"11,900.00",0.00,"2,400.00","6,075.00","20,375.00"
Sep-2023,"14,000.00",0.00,"2,400.00","6,750.00","23,150.00"
Oct-2023,"19,800.00",0.00,"2,400.00","6,750.00","28,950.00"


## **Calculating Finance Costs**

In [43]:
details_of_short_term_borrowings = helper.columns_to_snake_case(
    details_of_short_term_borrowing
)
details_of_long_term_borrowings = helper.columns_to_snake_case(
    details_of_long_term_borrowing
)

In [44]:
long_term_borrowings_schedules = borrowings.calculate_borrowings_schedules(details_of_long_term_borrowings)
long_term_borrowings_schedules.keys()

dict_keys(['interest_payments', 'capital_repayments'])

In [45]:
short_term_borrowings_schedules = borrowings.calculate_borrowings_schedules(details_of_short_term_borrowings)
short_term_borrowings_schedules.keys()

dict_keys(['interest_payments', 'capital_repayments'])

In [46]:
capital_repayment_borrowings_df = pd.concat(
    [
        long_term_borrowings_schedules["capital_repayments"],
        short_term_borrowings_schedules["capital_repayments"],
    ],
).fillna(0)

capital_repayment_borrowings_df.loc['total'] = capital_repayment_borrowings_df.sum()

In [47]:
finance_costs_df = pd.concat(
    [
        long_term_borrowings_schedules["interest_payments"],
        short_term_borrowings_schedules["interest_payments"],
    ],
).fillna(0)


finance_costs_df.loc["total"] = finance_costs_df.sum()
finance_costs_df

,May-2022,Jun-2022,Jul-2022,Aug-2022,Sep-2022,Oct-2022,Nov-2022,Dec-2022,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025,Jul-2025,Aug-2025,Sep-2025,Oct-2025,Nov-2025,Dec-2025,Jan-2026,Feb-2026,Mar-2026,Apr-2026,May-2026,Jun-2026,Jul-2026,Aug-2026,Sep-2026,Oct-2026,Nov-2026,Dec-2026,Jan-2027,Feb-2027,Mar-2027,Apr-2027,May-2027,Jun-2027,Jul-2027,Aug-2027,Sep-2027,Oct-2027,Nov-2027,Dec-2027
Grandre,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,"2,400.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
MINERVA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZAM-CARGO,0.00,0.00,0.00,0.00,"1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67","1,516.67",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZLA-Cargo,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"12,457.50",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LION INSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,000.00",0.00,0.00,0.00,0.00,0.00,"2,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LION INSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,784.00",0.00,0.00,0.00,0.00,0.00,"2,784.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BERAMASAMBA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,900.00,0.00,0.00,0.00,0.00,0.00,900.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LIFE ASSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,832.41","1,806.04","1,779.54","1,752.91","1,726.16","1,699.27","1,672.25","1,645.10","1,617.82","1,590.40","1,562.85","1,535.17","1,507.35","1,479.40","1,451.31","1,423.08","1,394.72","1,366.22","1,337.58","1,308.80","1,279.88","1,250.82","1,221.61","1,192.27","1,162.78","1,133.15","1,103.38","1,073.46","1,043.39","1,013.18",982.82,952.32,921.66,890.86,859.90,828.80,797.54,766.13,734.57,702.86,670.99,638.96,606.78,574.45,541.95,509.30,476.49,443.52,410.39,377.09,343.64,310.02,276.24,242.29,208.18,173.90,139.46,104.85,70.0

## **Income Statement**

In [59]:
income_statement_df = income_statement.generate_income_statement_template(
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Revenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bonus Provision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Calculate Variable Expenses**

In [61]:
uncertain_expenses = expenses.calculate_uncertain_expenses(
    expenses_uncertain=expenses_uncertain,
    other_parameters=other_parameters,
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)
uncertain_expenses

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Travel Costs,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Entertainment,925.21,943.71,962.58,981.84,"1,001.47","1,021.50","1,041.93","1,062.77","1,084.03","1,105.71","1,127.82","1,150.38"
Marketing Costs,"4,490.05","4,579.85","4,671.44","4,764.87","4,860.17","4,957.37","5,056.52","5,157.65","5,260.80","5,366.02","5,473.34","5,582.81"
Group Marketing Costs,"9,898.50","10,096.47","10,298.40","10,504.37","10,714.46","10,928.75","11,147.32","11,370.27","11,597.67","11,829.63","12,066.22","12,307.54"
Rental Costs,"1,199.94","1,223.94","1,248.41","1,273.38","1,298.85","1,324.83","1,351.32","1,378.35","1,405.92","1,434.04","1,462.72","1,491.97"
Subscriptions,585.04,596.74,608.67,620.85,633.26,645.93,658.85,672.03,685.47,699.18,713.16,727.42
Insurance,"2,458.87","2,508.05","2,558.21","2,609.37","2,661.56","2,714.79","2,769.08","2,824.47","2,880.96","2,938.57","2,997.35","3,057.29"
Repairs And Maintenance,"1,116.51","1,138.84","1,161.62","1,184.85","1,208.55","1,232.72","1,257.38","1,282.52","1,308.17","1,334.34","1,361.02","1,388.24"
Utilities,"1,412.25","1,440.49","1,469.30","1,498.69","1,528.66","1,559.23","1,590.42","1,622.23","1,654.67","1,687.76","1,721.52","1,755.95"
Stationery,822.22,838.67,855.44,872.55,890.00,907.80,925.96,944.48,963.37,982.63,"1,002.29","1,022.33"


### **Calculate Change In Provision For Credit Loss**

In [62]:
change_in_provision_for_credit_loss = (
    expenses.calculate_change_in_provision_for_credit_loss(
        provision_for_credit_loss=provision_for_credit_loss_for_all_new_disbursements['total'],
        provision_for_credit_loss_opening_balances=float(
            opening_balances["PROVISION_FOR_CREDIT_LOSS"].iat[0]
        ),
        start_date=start_date, 
        months_to_forecast=months_to_forecast
    )
)

change_in_provision_for_credit_loss

Jan-2023    9,960.00
Feb-2023    2,130.00
Mar-2023    2,190.00
Apr-2023   -1,880.00
May-2023      320.00
Jun-2023    1,620.00
Jul-2023    1,970.00
Aug-2023    4,065.00
Sep-2023    2,775.00
Oct-2023    5,800.00
Nov-2023    2,700.00
Dec-2023        0.00
Name: total, dtype: float64

### **Calculate Business Acquisition**

In [63]:
business_acquisition = expenses.calculate_business_acqusition(
    business_acquisition_percent=disbursement_parameters.loc['BUSINESS_ACQUISITION_PERCENT'],
    agent_contribution_percent=disbursement_parameters.loc['AGENT_CONTRIBUTION_PERCENT'],
    consumer_ssb_disbursements=new_disbursements_df['consumer_ssb_disbursements'],
    consumer_pvt_disbursements=new_disbursements_df['consumer_pvt_disbursements'],
)
business_acquisition

Jan-2023   1,620.00
Feb-2023   2,025.00
Mar-2023   2,484.00
Apr-2023   2,760.00
May-2023   3,312.00
Jun-2023   3,366.00
Jul-2023   3,927.00
Aug-2023   4,207.50
Sep-2023   4,275.00
Oct-2023   4,275.00
Nov-2023   4,275.00
Dec-2023   4,275.00
dtype: float64

### **Insert Revenue**

In [65]:
income_statement_df = income_statement.insert_revenue(
    income_statement=income_statement_df,
    interest_income=total_interest_income,
    other_income=other_income_df['total'],
)

income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Other Income,"22,143.37","35,378.37","53,340.94","66,753.73","77,098.65","90,882.76","95,790.73","104,609.60","114,527.68","138,794.07","164,677.33","189,308.55"
Total Revenue,"71,728.97","124,936.35","195,633.03","247,017.49","280,517.52","309,435.11","329,117.96","355,805.98","398,187.90","467,863.00","562,379.73","658,966.96"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bonus Provision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Static And Variable Inputs**

In [67]:
income_statement_df = income_statement.insert_expenses(
    income_statement=income_statement_df,
    expenses_certain=expenses_certain,
    uncertain_expenses=uncertain_expenses,
)

income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Other Income,"22,143.37","35,378.37","53,340.94","66,753.73","77,098.65","90,882.76","95,790.73","104,609.60","114,527.68","138,794.07","164,677.33","189,308.55"
Total Revenue,"71,728.97","124,936.35","195,633.03","247,017.49","280,517.52","309,435.11","329,117.96","355,805.98","398,187.90","467,863.00","562,379.73","658,966.96"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,0.00,58.39,0.00,296.32,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
Bonus Provision,"4,224.11","3,673.14","3,128.97","2,484.77","1,408.04",603.44,640.02,761.47,"3,071.99","2,929.90","2,901.54","2,837.47"


### **Insert Salaries and Pensions And Statutory Contributions**

In [68]:
income_statement_df = income_statement.insert_salaries_and_pensions_and_statutory_contributions(
    income_statement=income_statement_df,
    salaries_and_pension_and_statutory_contributions_df=salaries_and_pension_and_statutory_contributions_df,
)

income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Other Income,"22,143.37","35,378.37","53,340.94","66,753.73","77,098.65","90,882.76","95,790.73","104,609.60","114,527.68","138,794.07","164,677.33","189,308.55"
Total Revenue,"71,728.97","124,936.35","195,633.03","247,017.49","280,517.52","309,435.11","329,117.96","355,805.98","398,187.90","467,863.00","562,379.73","658,966.96"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"26,600.00","28,670.00","31,400.00","31,200.00","33,140.00","34,580.00","37,450.00","40,475.00","42,650.00","45,950.00","47,300.00","47,300.00"
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,0.00,58.39,0.00,296.32,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
Bonus Provision,"4,224.11","3,673.14","3,128.97","2,484.77","1,408.04",603.44,640.02,761.47,"3,071.99","2,929.90","2,901.54","2,837.47"


### **Insert Depreciation**

In [69]:
income_statement_df = income_statement.insert_depreciation(
    income_statement=income_statement_df,
    depreciation=depreciations_and_nbvs["dpns"]["total"],
)

### **Insert Change in Provision For Credit Loss**

In [70]:
income_statement_df = income_statement.insert_credit_loss_provision(
    income_statement=income_statement_df, 
    change_in_provision_for_credit_loss=change_in_provision_for_credit_loss,
)

In [72]:
provision_for_credit_loss = provision_for_credit_loss_for_all_new_disbursements["total"]
provision_for_credit_loss

Jan-2023    9,960.00
Feb-2023   12,090.00
Mar-2023   14,280.00
Apr-2023   12,400.00
May-2023   12,720.00
Jun-2023   14,340.00
Jul-2023   16,310.00
Aug-2023   20,375.00
Sep-2023   23,150.00
Oct-2023   28,950.00
Nov-2023   31,650.00
Dec-2023   31,650.00
Name: total, dtype: float64

### **Insert Business Acquisition**

In [73]:
income_statement_df = income_statement.insert_business_acquisition(
    income_statement=income_statement_df, business_acquisition=business_acquisition
)

### **Aggregations**

In [74]:
income_statement_df = income_statement.aggregate_staff_costs(income_statement_df)
income_statement_df = income_statement.aggregate_travel_and_entertainment(income_statement_df)
income_statement_df = income_statement.aggregate_marketing_and_public_relations(income_statement_df)
income_statement_df = income_statement.aggregate_office_costs(income_statement_df)
income_statement_df = income_statement.aggregate_professional_fees(income_statement_df)
income_statement_df = income_statement.aggregate_communication_costs(income_statement_df)
income_statement_df = income_statement.aggregate_motor_vehicle_costs(income_statement_df)
income_statement_df = income_statement.aggregate_other_costs(income_statement_df)
income_statement_df = income_statement.aggregate_investment_income(income_statement_df)
income_statement_df = income_statement.aggregate_finance_costs(income_statement_df)

### **Calculating Total Expenses and EBIDTA**

In [75]:
income_statement_df = income_statement.calculate_total_expenses(income_statement_df)
income_statement_df = income_statement.calculate_ebidta(income_statement_df)

### **Insert Finance Cost**

In [76]:
income_statement_df.loc["Finance Costs"] = finance_costs_df.loc["total"]
income_statement_df.loc["Finance Costs"] = income_statement_df.loc[
    "Finance Costs"
].fillna(0)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Other Income,"22,143.37","35,378.37","53,340.94","66,753.73","77,098.65","90,882.76","95,790.73","104,609.60","114,527.68","138,794.07","164,677.33","189,308.55"
Total Revenue,"71,728.97","124,936.35","195,633.03","247,017.49","280,517.52","309,435.11","329,117.96","355,805.98","398,187.90","467,863.00","562,379.73","658,966.96"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"26,600.00","28,670.00","31,400.00","31,200.00","33,140.00","34,580.00","37,450.00","40,475.00","42,650.00","45,950.00","47,300.00","47,300.00"
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,0.00,58.39,0.00,296.32,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
Bonus Provision,"4,224.11","3,673.14","3,128.97","2,484.77","1,408.04",603.44,640.02,761.47,"3,071.99","2,929.90","2,901.54","2,837.47"


### **Aggregate Finance Cost**

In [77]:
income_statement_df = income_statement.aggregate_finance_costs(income_statement_df)

### **Calculate Profit Before Tax**

In [78]:
income_statement_df = income_statement.calculate_profit_before_tax(income_statement_df)

### **Calculate Tax**

In [82]:
income_statement_df = income_statement.calculate_tax(
    income_statement=income_statement_df, tax_rate=other_parameters.loc["TAX_RATE"]
)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Other Income,"22,143.37","35,378.37","53,340.94","66,753.73","77,098.65","90,882.76","95,790.73","104,609.60","114,527.68","138,794.07","164,677.33","189,308.55"
Total Revenue,"71,728.97","124,936.35","195,633.03","247,017.49","280,517.52","309,435.11","329,117.96","355,805.98","398,187.90","467,863.00","562,379.73","658,966.96"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"26,600.00","28,670.00","31,400.00","31,200.00","33,140.00","34,580.00","37,450.00","40,475.00","42,650.00","45,950.00","47,300.00","47,300.00"
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,0.00,58.39,0.00,296.32,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
Bonus Provision,"4,224.11","3,673.14","3,128.97","2,484.77","1,408.04",603.44,640.02,761.47,"3,071.99","2,929.90","2,901.54","2,837.47"


## **Direct Cashflow**

In [83]:
direct_cashflow_df = direct_cashflow.generate_direct_cashflow_template(
    start_date=start_date, months_to_forecast=months_to_forecast
)

direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
DIRECT_CASHFLOW_STATEMENT,,,,,,,,,,,,
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Receipts From Receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sale Of Other Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Issue Of Equity And Intercompany Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Available Direct Cashflow Items**

#### **From Parameters**

In [84]:
direct_cashflow_df.loc[
    "Receipts From Receivables"
] = helper.change_period_index_to_strftime(
    helper.add_series(
        [
            other_parameters.loc["RECEIPTS_FROM_OTHER_RECEIVABLES"],
            other_parameters.loc["RECEIPTS_FROM_TRADE_RECEIVABLES"],
            other_parameters.loc["RECEIPTS_FROM_INTERGROUP_RECEIVABLES"],
        ]
    )
)

direct_cashflow_df.loc[
    "Purchase Of Inventory"
] = helper.change_period_index_to_strftime(other_parameters.loc["NEW_INVENTORY"])
direct_cashflow_df.loc[
    "Payments To Payables"
] = -helper.change_period_index_to_strftime(
    helper.add_series(
        [
            other_parameters.loc["PAYMENTS_TO_TRADE_PAYABLES"],
            other_parameters.loc["PAYMENTS_TO_OTHER_PAYABLES"],
        ]
    )
)

direct_cashflow_df.loc["Dividend Paid"] = -helper.change_period_index_to_strftime(
    other_parameters.loc["DIVIDEND_PAID"]
)

#### **From Calculations/Income Statement**

In [85]:
direct_cashflow_df.loc['Interest Income'] = income_statement_df.loc['Interest Income']
direct_cashflow_df.loc['Other Income'] = income_statement_df.loc['Other Income']
direct_cashflow_df.loc['Interest Expense'] = -income_statement_df.loc['Finance Costs']
direct_cashflow_df.loc['Disbursements'] = -helper.change_period_index_to_strftime(new_disbursements_df['total'])

In [86]:
direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
DIRECT_CASHFLOW_STATEMENT,,,,,,,,,,,,
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Other Income,"22,143.37","35,378.37","53,340.94","66,753.73","77,098.65","90,882.76","95,790.73","104,609.60","114,527.68","138,794.07","164,677.33","189,308.55"
Receipts From Receivables,0.00,0.00,0.00,0.00,"1,187,638.92",0.00,0.00,0.00,0.00,0.00,0.00,0.00
Sale Of Other Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Issue Of Equity And Intercompany Loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Equity and Intercompany Loans**

In [87]:
direct_cashflow_df = direct_cashflow.add_equity_and_intercompany_loans(
    other_parameters=other_parameters, direct_cashflow_df=direct_cashflow_df
)

### **Other Assets**


In [88]:
direct_cashflow_df = direct_cashflow.add_other_assets(
    other_parameters=other_parameters, direct_cashflow_df=direct_cashflow_df
)

### **Tax Paid**

In [89]:
tax_schedule = direct_cashflow.generate_tax_schedule(
    taxation=income_statement_df.loc["Taxation"],
    opening_balance=opening_balances["DEFERED_TAXATION"].iat[0],
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)

tax_schedule

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,0,0,0,0,0,0,0,0,0,0,0,0
Tax Charged,0,0,0,0,0,0,0,0,0,0,0,0
Tax Paid,0,0,0,0,0,0,0,0,0,0,0,0
Closing Balance,0,0,0,0,0,0,0,0,0,0,0,0


In [90]:
direct_cashflow_df.loc['Tax Paid'] = tax_schedule.loc['Tax Paid']

### **Calculate Operating Expenses**

In [91]:
operating_expenses = direct_cashflow.calculate_operating_expenses(income_statement=income_statement_df)
operating_expenses

Jan-2023    83,277.45
Feb-2023    90,699.66
Mar-2023    93,674.00
Apr-2023    96,854.74
May-2023   101,816.00
Jun-2023   106,060.23
Jul-2023   121,645.99
Aug-2023   123,376.91
Sep-2023   127,663.98
Oct-2023   134,312.08
Nov-2023   137,061.21
Dec-2023   136,478.88
dtype: object

### **Insert Operating Expenses**

In [92]:
direct_cashflow_df.loc['Operating Expenses']  = -operating_expenses
direct_cashflow_df


,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
DIRECT_CASHFLOW_STATEMENT,,,,,,,,,,,,
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Other Income,"22,143.37","35,378.37","53,340.94","66,753.73","77,098.65","90,882.76","95,790.73","104,609.60","114,527.68","138,794.07","164,677.33","189,308.55"
Receipts From Receivables,0.00,0.00,0.00,0.00,"1,187,638.92",0.00,0.00,0.00,0.00,0.00,0.00,0.00
Sale Of Other Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,0.00,0.00,0.00,0.00,0.00,0.00,61.99,98.62,775.08,0.00,0.00,0.00


### **Calculate Capital Expenses**

In [93]:
capital_expenses = direct_cashflow.calculate_capital_expenses(
    details_of_assets=details_of_assets,
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)

capital_expenses

Jan-2023    1,573,735.57
Feb-2023    3,111,295.81
Mar-2023      123,641.59
Apr-2023            0.00
May-2023            0.00
Jun-2023   13,639,154.03
Jul-2023   88,479,591.95
Aug-2023   26,054,584.74
Sep-2023            0.00
Oct-2023            0.00
Nov-2023            0.00
Dec-2023            0.00
Name: book_value, dtype: float64

### **Insert Capital Expenses**

In [94]:
direct_cashflow_df.loc['Capital Expenses'] = -capital_expenses

### **Calculate Direct Cashflow Borrowings**

In [95]:
long_and_short_term_borrowing_df = (
    direct_cashflow.calculate_long_and_short_term_borrowing_for_direct_cashflow(
        details_of_long_term_borrowing=details_of_long_term_borrowing,
        details_of_short_term_borrowing=details_of_short_term_borrowing,
        start_date=start_date,
        months_to_forecast=months_to_forecast,
    )
)

long_and_short_term_borrowing_df

,long_term_borrowing,short_term_borrowing,total
Jan-2023,0,"450,000.00","450,000.00"
Feb-2023,0,0.00,0.00
Mar-2023,30000,"75,000.00","105,000.00"
Apr-2023,30000,"138,813.77","168,813.77"
May-2023,0,0.00,0.00
Jun-2023,0,0.00,0.00
Jul-2023,0,0.00,0.00
Aug-2023,0,0.00,0.00
Sep-2023,0,0.00,0.00
Oct-2023,0,0.00,0.00


### **Insert Direct Cashflow Borrowing**

In [96]:
direct_cashflow_df.loc['Short Term Borrowing'] = long_and_short_term_borrowing_df['short_term_borrowing']
direct_cashflow_df.loc['Long Term Borrowing'] = long_and_short_term_borrowing_df['long_term_borrowing']

In [97]:
direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
DIRECT_CASHFLOW_STATEMENT,,,,,,,,,,,,
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,"450,000.00",0.00,"75,000.00","138,813.77",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Long Term Borrowing,0.00,0.00,"30,000.00","30,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Other Income,"22,143.37","35,378.37","53,340.94","66,753.73","77,098.65","90,882.76","95,790.73","104,609.60","114,527.68","138,794.07","164,677.33","189,308.55"
Receipts From Receivables,0.00,0.00,0.00,0.00,"1,187,638.92",0.00,0.00,0.00,0.00,0.00,0.00,0.00
Sale Of Other Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,0.00,0.00,0.00,0.00,0.00,0.00,61.99,98.62,775.08,0.00,0.00,0.00


### **Calculate Capital Repayments**

In [98]:
capital_repayment = helper.add_series(
[existing_loans_schedules['capital_repayments'].sum()
    , capital_repayment_new_disbursements_df['total']]
)

capital_repayment.loc[helper.generate_columns(start_date,months_to_forecast)]

Jan-2023   147,567.56
Feb-2023   205,768.44
Mar-2023   296,750.68
Apr-2023   400,274.83
May-2023   418,839.08
Jun-2023   498,022.14
Jul-2023   560,825.61
Aug-2023   594,419.03
Sep-2023   587,974.83
Oct-2023   641,248.56
Nov-2023   747,040.80
Dec-2023   872,221.87
dtype: float64

### **Insert Capital Repayment**

In [99]:
direct_cashflow_df.loc['Capital Repayment'] = capital_repayment

### **Calculate Capital Repayments on Borrowings**


In [100]:
capital_repayment_borrowings_df

,May-2022,Jun-2022,Jul-2022,Aug-2022,Sep-2022,Oct-2022,Nov-2022,Dec-2022,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025,Jul-2025,Aug-2025,Sep-2025,Oct-2025,Nov-2025,Dec-2025,Jan-2026,Feb-2026,Mar-2026,Apr-2026,May-2026,Jun-2026,Jul-2026,Aug-2026,Sep-2026,Oct-2026,Nov-2026,Dec-2026,Jan-2027,Feb-2027,Mar-2027,Apr-2027,May-2027,Jun-2027,Jul-2027,Aug-2027,Sep-2027,Oct-2027,Nov-2027,Dec-2027
Grandre,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"160,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
MINERVA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"15,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZAM-CARGO,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"364,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZLA-Cargo,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"124,575.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LION INSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"50,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LION INSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"92,800.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BERAMASAMBA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"30,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LIFE ASSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,417.73","5,444.10","5,470.60","5,497.23","5,523.98","5,550.87","5,577.89","5,605.04","5,632.32","5,659.74","5,687.29","5,714.97","5,742.79","5,770.74","5,798.83","5,827.06","5,855.42","5,883.92","5,912.56","5,941.34","5,970.26","5,999.32","6,028.53","6,057.87","6,087.36","6,116.99","6,146.76","6,176.68","6,206.75","6,236.96","6,267.32","6,297.82","6,328.48","6,359.28","6,390.24","6,421.34","6,452.60","6,484.01","6,515.57","6,547.28","6,579.15","6,611.18","6,643.36","6,675.69","6,708.19","6,740.84","6,773.65","6,806.62","6,839.76","6,873.05","6,906.50","6,940.12","6,973.90","7,007.85","7,041.96","7,076.24","7,110.68","7,145.29","7,180.07","7,215.02"
MASAWARA,80.78,81.17,81.57,81.96,82.36,82.76,83.17,83.57,83.98,84.39,84.80,85.21,85.63,86.04,86.46,86.88,87.31,87.73,88.16,88.59,89.02,89.45,

### **Insert Capital Repayment on New Borrowings**

In [101]:
direct_cashflow_df.loc[
    "Capital Repayment On Borrowings"
] = -capital_repayment_borrowings_df.loc["total"]

direct_cashflow_df.loc["Capital Repayment On Borrowings"] = direct_cashflow_df.loc[
    "Capital Repayment On Borrowings"
].fillna(0)

### **Calculate Direct Cashflow Totals**

In [102]:
direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
DIRECT_CASHFLOW_STATEMENT,,,,,,,,,,,,
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,"450,000.00",0.00,"75,000.00","138,813.77",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Long Term Borrowing,0.00,0.00,"30,000.00","30,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Capital Repayment,"147,567.56","205,768.44","296,750.68","400,274.83","418,839.08","498,022.14","560,825.61","594,419.03","587,974.83","641,248.56","747,040.80","872,221.87"
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Other Income,"22,143.37","35,378.37","53,340.94","66,753.73","77,098.65","90,882.76","95,790.73","104,609.60","114,527.68","138,794.07","164,677.33","189,308.55"
Receipts From Receivables,0.00,0.00,0.00,0.00,"1,187,638.92",0.00,0.00,0.00,0.00,0.00,0.00,0.00
Sale Of Other Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,0.00,0.00,0.00,0.00,0.00,0.00,61.99,98.62,775.08,0.00,0.00,0.00


In [103]:
direct_cashflow_df.loc["Total Cash Inflows"] = direct_cashflow_df.iloc[
    direct_cashflow_df.index.get_loc("CASH INFLOWS")
    + 1 : direct_cashflow_df.index.get_loc("Total Cash Inflows")
].sum()


direct_cashflow_df.loc["Total Cash Outflows"] = direct_cashflow_df.iloc[
    direct_cashflow_df.index.get_loc("CASH OUTFLOWS")
    + 1 : direct_cashflow_df.index.get_loc("Total Cash Outflows")
].sum()

direct_cashflow_df.loc["Net Increase/Decrease In Cash"] = (
    direct_cashflow_df.loc["Total Cash Inflows"]
    + direct_cashflow_df.loc["Total Cash Outflows"]
)

### **Calculate Opening and Closing Balances**

In [104]:
depreciations_and_nbvs['dpns'].sum(axis = 1)

Jan-2023   5,959,732.44
Feb-2023   5,959,732.44
Mar-2023   5,959,732.44
Apr-2023   5,959,732.44
May-2023   5,959,732.44
Jun-2023   5,959,732.44
Jul-2023   5,959,732.44
Aug-2023   5,959,732.44
Sep-2023   5,959,732.44
Oct-2023   5,959,732.44
Nov-2023   5,958,732.54
Dec-2023   5,958,048.17
dtype: float64

In [105]:
depreciations_and_nbvs['nbvs'].sum(axis = 1)

Jan-2023   295,484,695.90
Feb-2023   289,524,963.46
Mar-2023   283,565,231.01
Apr-2023   277,605,498.57
May-2023   271,645,766.13
Jun-2023   265,686,033.69
Jul-2023   259,726,301.25
Aug-2023   253,766,568.80
Sep-2023   247,806,836.36
Oct-2023   241,847,103.92
Nov-2023   235,894,478.54
Dec-2023   229,941,221.05
dtype: float64

In [106]:
direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
DIRECT_CASHFLOW_STATEMENT,,,,,,,,,,,,
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,"450,000.00",0.00,"75,000.00","138,813.77",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Long Term Borrowing,0.00,0.00,"30,000.00","30,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Capital Repayment,"147,567.56","205,768.44","296,750.68","400,274.83","418,839.08","498,022.14","560,825.61","594,419.03","587,974.83","641,248.56","747,040.80","872,221.87"
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Other Income,"22,143.37","35,378.37","53,340.94","66,753.73","77,098.65","90,882.76","95,790.73","104,609.60","114,527.68","138,794.07","164,677.33","189,308.55"
Receipts From Receivables,0.00,0.00,0.00,0.00,"1,187,638.92",0.00,0.00,0.00,0.00,0.00,0.00,0.00
Sale Of Other Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,0.00,0.00,0.00,0.00,0.00,0.00,61.99,98.62,775.08,0.00,0.00,0.00


In [107]:
tax_schedule

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,0,0,0,0,0,0,0,0,0,0,0,0
Tax Charged,0,0,0,0,0,0,0,0,0,0,0,0
Tax Paid,0,0,0,0,0,0,0,0,0,0,0,0
Closing Balance,0,0,0,0,0,0,0,0,0,0,0,0


In [108]:
direct_cashflow_df= direct_cashflow.calculate_opening_and_closing_balances_for_direct_cashflows(
direct_cashflow=direct_cashflow_df, 
cash_on_hand_opening_balance= opening_balances['CASH_ON_HAND'].iat[0]
)

direct_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
DIRECT_CASHFLOW_STATEMENT,,,,,,,,,,,,
CASH INFLOWS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,"450,000.00",0.00,"75,000.00","138,813.77",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Long Term Borrowing,0.00,0.00,"30,000.00","30,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Capital Repayment,"147,567.56","205,768.44","296,750.68","400,274.83","418,839.08","498,022.14","560,825.61","594,419.03","587,974.83","641,248.56","747,040.80","872,221.87"
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Other Income,"22,143.37","35,378.37","53,340.94","66,753.73","77,098.65","90,882.76","95,790.73","104,609.60","114,527.68","138,794.07","164,677.33","189,308.55"
Receipts From Receivables,0.00,0.00,0.00,0.00,"1,187,638.92",0.00,0.00,0.00,0.00,0.00,0.00,0.00
Sale Of Other Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Issue Of Equity And Intercompany Loans,0.00,0.00,0.00,0.00,0.00,0.00,61.99,98.62,775.08,0.00,0.00,0.00


### **Calculate imtt (2% Tax)**

In [109]:
income_statement_df.loc['2% Taxation']  = -direct_cashflow_df.loc['Total Cash Outflows']*imtt

## **Calculate Profit and Loss for the Period**


In [110]:
income_statement_df = income_statement.calculate_profit_or_loss_for_period(income_statement_df)
income_statement_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME_STATEMENT,,,,,,,,,,,,
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Other Income,"22,143.37","35,378.37","53,340.94","66,753.73","77,098.65","90,882.76","95,790.73","104,609.60","114,527.68","138,794.07","164,677.33","189,308.55"
Total Revenue,"71,728.97","124,936.35","195,633.03","247,017.49","280,517.52","309,435.11","329,117.96","355,805.98","398,187.90","467,863.00","562,379.73","658,966.96"
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,"26,600.00","28,670.00","31,400.00","31,200.00","33,140.00","34,580.00","37,450.00","40,475.00","42,650.00","45,950.00","47,300.00","47,300.00"
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,0.00,58.39,0.00,296.32,0.00,0.00,0.00,0.00,0.00,"1,000.00","1,000.00",0.00
Bonus Provision,"4,224.11","3,673.14","3,128.97","2,484.77","1,408.04",603.44,640.02,761.47,"3,071.99","2,929.90","2,901.54","2,837.47"


## **Loan Book**

In [111]:
loan_book_df = loan_book.generate_loan_book_template(
    start_date=start_date, months_to_forecast=months_to_forecast
)
loan_book_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New Disbursements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Repayments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Closing Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Loan Book Items**

In [112]:
capital_repayment_existing_loans = existing_loans_schedules['capital_repayments'].sum()
capital_repayment_existing_loans.loc[helper.generate_columns(start_date,months_to_forecast)]

Jan-2023   147,567.56
Feb-2023   144,585.73
Mar-2023   160,532.02
Apr-2023   172,443.92
May-2023   109,934.28
Jun-2023   100,375.41
Jul-2023    63,408.12
Aug-2023    59,655.62
Sep-2023    10,092.45
Oct-2023     6,747.14
Nov-2023     4,764.47
Dec-2023     2,982.53
dtype: float64

In [113]:
total_capital_repayments = loan_book.aggregate_new_and_existing_loans_capital_repayments(
    capital_repayments_new_disbursements_df = capital_repayment_new_disbursements_df,
    capital_repayments_existing_loans = capital_repayment_existing_loans,
    start_date = start_date,
    months_to_forecast = months_to_forecast,
)

total_capital_repayments

Jan-2023   147,567.56
Feb-2023   205,768.44
Mar-2023   296,750.68
Apr-2023   400,274.83
May-2023   418,839.08
Jun-2023   498,022.14
Jul-2023   560,825.61
Aug-2023   594,419.03
Sep-2023   587,974.83
Oct-2023   641,248.56
Nov-2023   747,040.80
Dec-2023   872,221.87
dtype: float64

In [114]:
# epening_balance_on_loan_book = existing_loans.loc[existing_loans['closing_balance']>0]['closing_balance'].sum()

loan_book_df = loan_book.insert_loan_book_items(
    loan_book=loan_book_df,
    opening_balance_on_loan_book=float(opening_balances["LOAN_BOOK"].iat[0]),
    total_interest_income=total_interest_income,
    total_capital_repayments=total_capital_repayments,
    disbursements=helper.change_period_index_to_strftime(new_disbursements_df["total"]),
)

loan_book_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,"2,806,038.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New Disbursements,"537,011.05","609,480.25","723,480.25","634,900.00","678,173.00","720,709.25","838,650.00","1,008,680.00","1,215,000.00","1,510,000.00","1,585,000.00","1,585,000.00"
Repayments,"-197,153.16","-295,326.43","-439,042.77","-580,538.59","-622,257.95","-716,574.49","-794,152.83","-845,615.41","-871,635.05","-970,317.49","-1,144,743.19","-1,341,880.28"
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Closing Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Calculate Opening and Closing Balances for Loan Book**

In [115]:
loan_book_df = helper.calculate_opening_and_closing_balances(loan_book_df)
loan_book_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,"2,806,038.00","3,195,481.49","3,599,193.30","4,025,922.87","4,260,548.04","4,519,881.97","4,742,569.08","5,020,393.47","5,434,654.45","6,061,679.61","6,930,431.06","7,768,390.26"
New Disbursements,"537,011.05","609,480.25","723,480.25","634,900.00","678,173.00","720,709.25","838,650.00","1,008,680.00","1,215,000.00","1,510,000.00","1,585,000.00","1,585,000.00"
Repayments,"-197,153.16","-295,326.43","-439,042.77","-580,538.59","-622,257.95","-716,574.49","-794,152.83","-845,615.41","-871,635.05","-970,317.49","-1,144,743.19","-1,341,880.28"
Interest Income,"49,585.60","89,557.99","142,292.09","180,263.76","203,418.87","218,552.35","233,327.22","251,196.38","283,660.21","329,068.93","397,702.40","469,658.41"
Closing Balance,"3,195,481.49","3,599,193.30","4,025,922.87","4,260,548.04","4,519,881.97","4,742,569.08","5,020,393.47","5,434,654.45","6,061,679.61","6,930,431.06","7,768,390.26","8,481,168.40"


## **Balance Sheet**


In [116]:
balance_sheet_df = balance_sheet.generate_balance_sheet_template(
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)

balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Intangible Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Subsidiaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Associates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment Properties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Equity Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Money Market Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Insert Available Balance Sheet Items**

In [117]:
depreciations_and_nbvs.keys()

dict_keys(['nbvs', 'dpns'])

In [118]:
balance_sheet_df.loc["Property Plant And Equipment"] = depreciations_and_nbvs["nbvs"][
    "total"
]

balance_sheet_df.loc["Loan Book"] = loan_book_df.loc["Closing Balance"]

balance_sheet_df.loc["Cash On Hand"] = direct_cashflow_df.loc["Closing Balance"]

balance_sheet_df.loc[
    "Provisions"
] = provision_for_credit_loss_for_all_new_disbursements["total"]


balance_sheet_df.loc["Provision For Taxation"] = helper.change_period_index_to_strftime(
    other_parameters.loc["PROVISION_FOR_TAX"]
)

balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,"147,742,347.95","144,762,481.73","141,782,615.51","138,802,749.29","135,822,883.07","132,843,016.84","129,863,150.62","126,883,284.40","123,903,418.18","120,923,551.96","117,947,239.27","114,970,610.52"
Intangible Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Subsidiaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Associates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment Properties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Equity Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Money Market Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Generate Borrowings Schedules**

In [119]:
short_term_loans_schedules = balance_sheet.calculate_short_term_loans_schedules(
    long_and_short_term_borrowing_df=long_and_short_term_borrowing_df,
    capital_repayment_on_borrowings_df=short_term_borrowings_schedules['capital_repayments'].sum(),
    opening_balances=opening_balances,
    start_date=start_date, 
    months_to_forecast=months_to_forecast
)

short_term_loans_schedules

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,0,"450,000.00","450,000.00","516,892.48","647,509.72","639,223.20","630,845.70","622,376.23","613,813.77","613,813.77","613,813.77","613,813.77"
Borrowings,"450,000.00",0.00,"75,000.00","138,813.77",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Repayments,0,0,"-8,107.52","-8,196.53","-8,286.52","-8,377.50","-8,469.47","-8,562.46",0,0,0,"-740,383.84"
Closing Balance,"450,000.00","450,000.00","516,892.48","647,509.72","639,223.20","630,845.70","622,376.23","613,813.77","613,813.77","613,813.77","613,813.77","-126,570.07"


In [120]:
capital_repayment_borrowings_df

,May-2022,Jun-2022,Jul-2022,Aug-2022,Sep-2022,Oct-2022,Nov-2022,Dec-2022,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024,Feb-2024,Mar-2024,Apr-2024,May-2024,Jun-2024,Jul-2024,Aug-2024,Sep-2024,Oct-2024,Nov-2024,Dec-2024,Jan-2025,Feb-2025,Mar-2025,Apr-2025,May-2025,Jun-2025,Jul-2025,Aug-2025,Sep-2025,Oct-2025,Nov-2025,Dec-2025,Jan-2026,Feb-2026,Mar-2026,Apr-2026,May-2026,Jun-2026,Jul-2026,Aug-2026,Sep-2026,Oct-2026,Nov-2026,Dec-2026,Jan-2027,Feb-2027,Mar-2027,Apr-2027,May-2027,Jun-2027,Jul-2027,Aug-2027,Sep-2027,Oct-2027,Nov-2027,Dec-2027
Grandre,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"160,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
MINERVA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"15,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZAM-CARGO,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"364,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZLA-Cargo,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"124,575.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LION INSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"50,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LION INSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"92,800.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
BERAMASAMBA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"30,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ZIMNAT LIFE ASSURANCE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"5,417.73","5,444.10","5,470.60","5,497.23","5,523.98","5,550.87","5,577.89","5,605.04","5,632.32","5,659.74","5,687.29","5,714.97","5,742.79","5,770.74","5,798.83","5,827.06","5,855.42","5,883.92","5,912.56","5,941.34","5,970.26","5,999.32","6,028.53","6,057.87","6,087.36","6,116.99","6,146.76","6,176.68","6,206.75","6,236.96","6,267.32","6,297.82","6,328.48","6,359.28","6,390.24","6,421.34","6,452.60","6,484.01","6,515.57","6,547.28","6,579.15","6,611.18","6,643.36","6,675.69","6,708.19","6,740.84","6,773.65","6,806.62","6,839.76","6,873.05","6,906.50","6,940.12","6,973.90","7,007.85","7,041.96","7,076.24","7,110.68","7,145.29","7,180.07","7,215.02"
MASAWARA,80.78,81.17,81.57,81.96,82.36,82.76,83.17,83.57,83.98,84.39,84.80,85.21,85.63,86.04,86.46,86.88,87.31,87.73,88.16,88.59,89.02,89.45,

In [121]:
long_and_short_term_borrowing_df

,long_term_borrowing,short_term_borrowing,total
Jan-2023,0,"450,000.00","450,000.00"
Feb-2023,0,0.00,0.00
Mar-2023,30000,"75,000.00","105,000.00"
Apr-2023,30000,"138,813.77","168,813.77"
May-2023,0,0.00,0.00
Jun-2023,0,0.00,0.00
Jul-2023,0,0.00,0.00
Aug-2023,0,0.00,0.00
Sep-2023,0,0.00,0.00
Oct-2023,0,0.00,0.00


In [122]:
long_term_loans_schedules = balance_sheet.calculate_long_term_loans_schedules(
    long_and_short_term_borrowing_df=long_and_short_term_borrowing_df,
    capital_repayment_on_borrowings_df=long_term_borrowings_schedules['capital_repayments'].sum(),
    opening_balances=opening_balances,
    start_date=start_date, 
    months_to_forecast=months_to_forecast
)

long_term_loans_schedules

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,0,"-5,501.71","-11,030.19","13,414.41","36,732.63","30,011.60","23,251.06","16,450.75","9,610.39","2,729.72","-4,191.53","-11,153.65"
Borrowings,0,0,30000,30000,0,0,0,0,0,0,0,0
Repayments,"-5,501.71","-5,528.49","-5,555.40","-6,681.79","-6,721.03","-6,760.54","-6,800.31","-6,840.35","-6,880.67","-6,921.26","-6,962.12","-274,378.26"
Closing Balance,"-5,501.71","-11,030.19","13,414.41","36,732.63","30,011.60","23,251.06","16,450.75","9,610.39","2,729.72","-4,191.53","-11,153.65","-285,531.91"


### **Insert Borrowings and Loans**


In [123]:
balance_sheet_df.loc["Loans"] = long_term_loans_schedules.loc["Closing Balance"]
balance_sheet_df.loc["Borrowings"] = short_term_loans_schedules.loc["Closing Balance"]

### **Generate Receivables and Payables Schedules**

In [124]:
trade_receivables_schedule = balance_sheet.generate_receivables_schedule(
    opening_receivables=opening_balances["TRADE_RECEIVABLES"].iat[0],
    receipts_from_receivables=other_parameters.loc["RECEIPTS_FROM_TRADE_RECEIVABLES"],
    new_receivables=other_parameters.loc["NEW_TRADE_RECEIVABLES"],
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)

other_receivables_schedule = balance_sheet.generate_receivables_schedule(
    opening_receivables=opening_balances["OTHER_RECEIVABLES"].iat[0],
    receipts_from_receivables=other_parameters.loc["RECEIPTS_FROM_OTHER_RECEIVABLES"],
    new_receivables=other_parameters.loc["NEW_OTHER_RECEIVABLES"],
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)

intergroup_receivables_schedule = balance_sheet.generate_receivables_schedule(
    opening_receivables=opening_balances["INTERGROUP_RECEIVABLES"].iat[0],
    receipts_from_receivables=other_parameters.loc["RECEIPTS_FROM_INTERGROUP_RECEIVABLES"],
    new_receivables=other_parameters.loc["NEW_INTERGROUP_RECEIVABLES"],
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)

trade_payables_schedule = balance_sheet.generate_payables_schedule(
    opening_payables=opening_balances["TRADE_PAYABLES"].iat[0],
    payments_to_payables=other_parameters.loc["PAYMENTS_TO_TRADE_PAYABLES"],
    new_payables=other_parameters.loc["NEW_TRADE_PAYABLES"],
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)

other_payables_schedule = balance_sheet.generate_payables_schedule(
    opening_payables=opening_balances["OTHER_PAYABLES"].iat[0],
    payments_to_payables=other_parameters.loc["PAYMENTS_TO_OTHER_PAYABLES"],
    new_payables=other_parameters.loc["NEW_OTHER_PAYABLES"],
    months_to_forecast=months_to_forecast,
    start_date=start_date,
)


balance_sheet_df.loc["Trade Payables"] = trade_payables_schedule.loc["Closing Balance"]
balance_sheet_df.loc["Other Payables"] = other_payables_schedule.loc["Closing Balance"]

balance_sheet_df.loc["Trade Receivables"] = trade_receivables_schedule.loc[
    "Closing Balance"
]

balance_sheet_df.loc["Other Receivables"] = other_receivables_schedule.loc[
    "Closing Balance"
]

balance_sheet_df.loc["Intergroup Receivables"] = intergroup_receivables_schedule.loc[
    "Closing Balance"
]


balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,"147,742,347.95","144,762,481.73","141,782,615.51","138,802,749.29","135,822,883.07","132,843,016.84","129,863,150.62","126,883,284.40","123,903,418.18","120,923,551.96","117,947,239.27","114,970,610.52"
Intangible Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Subsidiaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment In Associates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Investment Properties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Equity Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Money Market Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
balance_sheet_df.loc['Deferred Taxation'] = tax_schedule.loc['Closing Balance']

In [126]:
inventories_schedule = balance_sheet.generate_inventories_schedule(
    opening_inventories=opening_balances["INVENTORIES"].iat[0],
    new_inventories=other_parameters.loc["NEW_INVENTORY"],
    inventories_used=other_parameters.loc["INVENTORY_USED"],
    start_date=start_date,
    months_to_forecast=months_to_forecast,
)

inventories_schedule

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,0,0,0,0,0,0,0,0,0,0,0,0
New Inventories,0,0,0,0,0,0,0,0,0,0,0,0
Inventories Used,0,0,0,0,0,0,0,0,0,0,0,0
Closing Balance,0,0,0,0,0,0,0,0,0,0,0,0


In [127]:
balance_sheet_df.loc['Inventories']  = inventories_schedule.loc['Closing Balance']

### **Equity and Intercompany Loans**


In [128]:
balance_sheet_df.loc["Issued Share Capital"] = helper.change_period_index_to_strftime(
    other_parameters.loc["SHARE_CAPITAL"].cumsum()
    + opening_balances["ISSUED_SHARE_CAPITAL"].iat[0]
)

balance_sheet_df.loc["Intercompany Loans"] = helper.change_period_index_to_strftime(
    other_parameters.loc["INTERCOMPANY_LOANS"].cumsum()
    + opening_balances["INTERCOMPANY_LOANS"].iat[0]
)


balance_sheet_df.loc["Share Premium"] = helper.change_period_index_to_strftime(
    other_parameters.loc["SHARE_PREMIUM"].cumsum() + opening_balances["SHARE_PREMIUM"].iat[0]
)

balance_sheet_df.loc[
    "Other Components Of Equity"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["OTHER_COMPONENTS_OF_EQUITY"].cumsum()
    + opening_balances["OTHER_COMPONENTS_OF_EQUITY"].iat[0]
)

balance_sheet_df.loc["Treasury Shares"] = helper.change_period_index_to_strftime(
    other_parameters.loc["TREASURY_SHARES"].cumsum()
    + opening_balances["TREASURY_SHARES"].iat[0]
)

balance_sheet_df.loc["Retained Earnings"] = (
    income_statement_df.loc["PROFIT/(LOSS) FOR PERIOD"]
    - helper.change_period_index_to_strftime(other_parameters.loc["DIVIDEND_PAID"])
).cumsum()

balance_sheet_df.loc["Capital And Reserves"] = balance_sheet_df.loc[
    "Issued Share Capital":"Retained Earnings"
].sum()

### **Other Assets**

In [129]:
balance_sheet_df.loc["Intangible Assets"] = helper.change_period_index_to_strftime(
    other_parameters.loc["INTANGIBLE_ASSETS"].cumsum()
    + opening_balances["INTANGIBLE_ASSETS"].iat[0]
)

balance_sheet_df.loc[
    "Investment In Subsidiaries"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["INVESTMENT_IN_SUBSIDIARIES"].cumsum()
    + opening_balances["INVESTMENT_IN_SUBSIDIARIES"].iat[0]
)

balance_sheet_df.loc[
    "Investment In Associates"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["INVESTMENT_IN_ASSOCIATES"].cumsum()
    + opening_balances["INVESTMENT_IN_ASSOCIATES"].iat[0]
)

balance_sheet_df.loc["Investment Properties"] = helper.change_period_index_to_strftime(
    other_parameters.loc["INVESTMENT_PROPERTIES"].cumsum()
    + opening_balances["INVESTMENT_PROPERTIES"].iat[0]
)

balance_sheet_df.loc["Equity Investments"] = helper.change_period_index_to_strftime(
    other_parameters.loc["EQUITY_INVESTMENTS"].cumsum()
    + opening_balances["EQUITY_INVESTMENTS"].iat[0]
)

balance_sheet_df.loc[
    "Long Term Money Market Investments"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["LONG_TERM_MONEY_MARKET_INVESTMENTS"].cumsum()
    + opening_balances["LONG_TERM_MONEY_MARKET_INVESTMENTS"].iat[0]
)

balance_sheet_df.loc[
    "Short Term Money Market Investments"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["SHORT_TERM_MONEY_MARKET_INVESTMENTS"].cumsum()
    + opening_balances["SHORT_TERM_MONEY_MARKET_INVESTMENTS"].iat[0]
)

balance_sheet_df.loc[
    "Loans To Related Entities"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["LOANS_TO_RELATED_ENTITIES"].cumsum()
    + opening_balances["LOANS_TO_RELATED_ENTITIES"].iat[0]
)

In [130]:
balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,"147,742,347.95","144,762,481.73","141,782,615.51","138,802,749.29","135,822,883.07","132,843,016.84","129,863,150.62","126,883,284.40","123,903,418.18","120,923,551.96","117,947,239.27","114,970,610.52"
Intangible Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Investment In Subsidiaries,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Investment In Associates,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Investment Properties,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Equity Investments,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Long Term Money Market Investments,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### **Calculate Totals**


In [131]:
balance_sheet_df = balance_sheet.sum_financial_statements_totals(balance_sheet_df)
balance_sheet_df = balance_sheet.calculate_final_balances(balance_sheet_df=balance_sheet_df)
balance_sheet_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_FINANCIAL_POSITION,,,,,,,,,,,,
ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NON CURRENT ASSETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Property Plant And Equipment,"147,742,347.95","144,762,481.73","141,782,615.51","138,802,749.29","135,822,883.07","132,843,016.84","129,863,150.62","126,883,284.40","123,903,418.18","120,923,551.96","117,947,239.27","114,970,610.52"
Intangible Assets,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Investment In Subsidiaries,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Investment In Associates,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Investment Properties,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Equity Investments,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Long Term Money Market Investments,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## **Indirect Cashflow**

In [132]:
statement_of_cashflow_df = (
    statement_of_cashflows.generate_statement_of_cashflow_template(
        start_date, months_to_forecast
    )
)

### **Insert Available Balances**

In [133]:
statement_of_cashflow_df.loc["Profit/(loss) Per I/S"] = income_statement_df.loc[
    "PROFIT/(LOSS) FOR PERIOD"
]
statement_of_cashflow_df.loc["Depreciation"] = income_statement_df.loc["Depreciation"]
statement_of_cashflow_df.loc["Dividend Paid"] = -helper.change_period_index_to_strftime(
    other_parameters.loc["DIVIDEND_PAID"]
)
statement_of_cashflow_df.loc[
    "Treasury Movements"
] = helper.change_period_index_to_strftime(other_parameters.loc["TREASURY_MOVEMENTS"])
statement_of_cashflow_df.loc[
    "Interest Expense Accrued"
] = helper.change_period_index_to_strftime(
    other_parameters.loc["INTEREST_EXPENSE_ACCRUED"]
)
statement_of_cashflow_df.loc[
    "Other Non-Cash Items"
] = helper.change_period_index_to_strftime(other_parameters.loc["OTHER_NON_CASH_ITEMS"])
statement_of_cashflow_df.loc["Interest Paid"] = -finance_costs_df.loc["total"]
statement_of_cashflow_df.loc["Tax Paid"] = tax_schedule.loc["Tax Paid"]
statement_of_cashflow_df.loc[
    "Repayment Of Borrowings"
] = -capital_repayment_borrowings_df.loc["total"]
statement_of_cashflow_df.loc["Purchase Of Fixed Assets"] = capital_expenses
statement_of_cashflow_df.loc["Increase/(Decrease) In Short Term Borrowings"] = (
    short_term_loans_schedules.loc["Closing Balance"]
    - short_term_loans_schedules.loc["Opening Balance"]
)
statement_of_cashflow_df.loc["Increase/(Decrease) In Long Term Borrowings"] = (
    long_term_loans_schedules.loc["Closing Balance"]
    - long_term_loans_schedules.loc["Opening Balance"]
)
statement_of_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_CASHFLOWS,,,,,,,,,,,,
Profit/(loss) Per I/S,"-3,031,001.00","-2,993,090.47","-2,948,513.02","-2,842,695.86","-2,819,483.39","-2,983,360.37","-3,673,283.06","-3,027,876.91","-2,784,207.42","-2,674,972.74","-2,580,313.52","-2,535,004.78"
Treasury Movements,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Adjustments For:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Interest Expense Accrued,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other Non-Cash Items,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cash From Operations Before WC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Working Capital Movements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(Increase)/Decrease In Receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
statement_of_cashflow_df.loc[
    "Cash From Operations Before WC"
] = statement_of_cashflow_df.iloc[
    1 : statement_of_cashflow_df.index.get_loc("Cash From Operations Before WC")
].sum()

In [135]:

statement_of_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_CASHFLOWS,,,,,,,,,,,,
Profit/(loss) Per I/S,"-3,031,001.00","-2,993,090.47","-2,948,513.02","-2,842,695.86","-2,819,483.39","-2,983,360.37","-3,673,283.06","-3,027,876.91","-2,784,207.42","-2,674,972.74","-2,580,313.52","-2,535,004.78"
Treasury Movements,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Adjustments For:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Interest Expense Accrued,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other Non-Cash Items,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cash From Operations Before WC,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Working Capital Movements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(Increase)/Decrease In Receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Working Capital Movements**

In [136]:
change_in_receivables = (
    other_receivables_schedule.loc["Closing Balance"]
    - other_receivables_schedule.loc["Opening Balance"]
    + trade_receivables_schedule.loc["Closing Balance"]
    - trade_receivables_schedule.loc["Opening Balance"]
    + intergroup_receivables_schedule.loc["Closing Balance"]
    - intergroup_receivables_schedule.loc["Opening Balance"]
)

change_in_receivables

Jan-2023            0.00
Feb-2023       91,225.40
Mar-2023       57,639.36
Apr-2023       58,279.67
May-2023   -1,187,638.92
Jun-2023       27,773.92
Jul-2023        1,728.17
Aug-2023        2,654.09
Sep-2023        3,425.38
Oct-2023        5,042.59
Nov-2023        5,990.38
Dec-2023       10,202.77
dtype: float64

In [137]:
change_in_payables = (
    trade_payables_schedule.loc["Closing Balance"]
    - trade_payables_schedule.loc["Opening Balance"]
    + other_payables_schedule.loc["Closing Balance"]
    - other_payables_schedule.loc["Opening Balance"]
)
change_in_payables

Jan-2023          0.00
Feb-2023    -13,404.72
Mar-2023    -68,786.66
Apr-2023    -80,963.15
May-2023   -466,687.36
Jun-2023     15,888.40
Jul-2023     19,217.95
Aug-2023     32,088.24
Sep-2023      3,425.38
Oct-2023      5,042.59
Nov-2023      5,990.38
Dec-2023     10,202.77
dtype: float64

In [138]:
change_in_loan_book_principle = loan_book_df.loc["New Disbursements":"Repayments"].sum()
change_in_loan_book_principle

Jan-2023   339,857.89
Feb-2023   314,153.82
Mar-2023   284,437.48
Apr-2023    54,361.41
May-2023    55,915.05
Jun-2023     4,134.76
Jul-2023    44,497.17
Aug-2023   163,064.59
Sep-2023   343,364.95
Oct-2023   539,682.51
Nov-2023   440,256.81
Dec-2023   243,119.72
dtype: object

In [139]:
change_in_loan_book_interest = loan_book_df.loc['Interest Income']
change_in_loan_book_interest

Jan-2023    49,585.60
Feb-2023    89,557.99
Mar-2023   142,292.09
Apr-2023   180,263.76
May-2023   203,418.87
Jun-2023   218,552.35
Jul-2023   233,327.22
Aug-2023   251,196.38
Sep-2023   283,660.21
Oct-2023   329,068.93
Nov-2023   397,702.40
Dec-2023   469,658.41
Name: Interest Income, dtype: object

In [140]:
borrowings_schedule = long_term_loans_schedules + short_term_loans_schedules
change_in_borrowings = borrowings_schedule.loc['Closing Balance'] - borrowings_schedule.loc['Opening Balance']
change_in_borrowings

Jan-2023      444,498.29
Feb-2023       -5,528.49
Mar-2023       91,337.08
Apr-2023      153,935.45
May-2023      -15,007.55
Jun-2023      -15,138.03
Jul-2023      -15,269.78
Aug-2023      -15,402.81
Sep-2023       -6,880.67
Oct-2023       -6,921.26
Nov-2023       -6,962.12
Dec-2023   -1,014,762.10
dtype: object

In [141]:
statement_of_cashflow_df.loc['(Increase)/Decrease In Receivables'] = change_in_receivables
statement_of_cashflow_df.loc['Increase/(Decrease) In Payables'] = change_in_payables
statement_of_cashflow_df.loc['(Increase)/Decrease In Loan Book (Principle)'] = change_in_loan_book_principle
statement_of_cashflow_df.loc['(Increase)/Decrease In Loan Book (Interest)'] = change_in_loan_book_interest
statement_of_cashflow_df.loc['Increase/(Decrease) In Borrowings'] = change_in_borrowings

In [142]:
statement_of_cashflow_df.loc["Cash From Operations After WC"] = (
    statement_of_cashflow_df.loc["Cash From Operations Before WC"]
    + statement_of_cashflow_df.iloc[
        statement_of_cashflow_df.index.get_loc("Working Capital Movements")
        + 1 : statement_of_cashflow_df.index.get_loc("Cash From Operations After WC")
    ].sum()
)

In [143]:
statement_of_cashflow_df.loc[
    "Net Cash Flow From Operations"
] = statement_of_cashflow_df.loc["Cash From Operations After WC":"Tax Paid"].sum()

In [144]:
statement_of_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_CASHFLOWS,,,,,,,,,,,,
Profit/(loss) Per I/S,"-3,031,001.00","-2,993,090.47","-2,948,513.02","-2,842,695.86","-2,819,483.39","-2,983,360.37","-3,673,283.06","-3,027,876.91","-2,784,207.42","-2,674,972.74","-2,580,313.52","-2,535,004.78"
Treasury Movements,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Adjustments For:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Interest Expense Accrued,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other Non-Cash Items,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cash From Operations Before WC,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Working Capital Movements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(Increase)/Decrease In Receivables,0.00,"91,225.40","57,639.36","58,279.67","-1,187,638.92","27,773.92","1,728.17","2,654.09","3,425.38","5,042.59","5,990.38","10,202.77"


### **Calculating Totals**

In [145]:
statement_of_cashflow_df = balance_sheet.sum_financial_statements_totals(statement_of_cashflow_df)


statement_of_cashflow_df.loc['Net Increase/(Decrease) In Cash'] = (
    statement_of_cashflow_df.loc["Net Cash Flow From Operations"]
    + statement_of_cashflow_df.loc["Cash Flow From Investing Activities"]
    + statement_of_cashflow_df.loc["Cash Flow From Financing Activities"]
)
statement_of_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_CASHFLOWS,,,,,,,,,,,,
Profit/(loss) Per I/S,"-3,031,001.00","-2,993,090.47","-2,948,513.02","-2,842,695.86","-2,819,483.39","-2,983,360.37","-3,673,283.06","-3,027,876.91","-2,784,207.42","-2,674,972.74","-2,580,313.52","-2,535,004.78"
Treasury Movements,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Adjustments For:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Interest Expense Accrued,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other Non-Cash Items,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cash From Operations Before WC,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Working Capital Movements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(Increase)/Decrease In Receivables,0.00,"91,225.40","57,639.36","58,279.67","-1,187,638.92","27,773.92","1,728.17","2,654.09","3,425.38","5,042.59","5,990.38","10,202.77"


### **Calculating Cash at End and Beginning of Period**

In [146]:
statement_of_cashflow_df = (
    statement_of_cashflows.calculate_cash_at_end_and_beginning_of_period(
        statement_of_cashflow_df=statement_of_cashflow_df,
        opening_balances=opening_balances,
    )
)

statement_of_cashflow_df

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
STATEMENT_OF_CASHFLOWS,,,,,,,,,,,,
Profit/(loss) Per I/S,"-3,031,001.00","-2,993,090.47","-2,948,513.02","-2,842,695.86","-2,819,483.39","-2,983,360.37","-3,673,283.06","-3,027,876.91","-2,784,207.42","-2,674,972.74","-2,580,313.52","-2,535,004.78"
Treasury Movements,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Adjustments For:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Depreciation,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Interest Expense Accrued,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Other Non-Cash Items,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Cash From Operations Before WC,"2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,866.22","2,979,366.27","2,979,024.08"
Working Capital Movements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(Increase)/Decrease In Receivables,0.00,"91,225.40","57,639.36","58,279.67","-1,187,638.92","27,773.92","1,728.17","2,654.09","3,425.38","5,042.59","5,990.38","10,202.77"
